# Homework 2 Solutions

BEE 4850/5850

## Overview

### Instructions

The goal of this homework assignment is to practice developing and
working with probability models for data.

-   Problem 1 asks you to fit a sea-level rise model using normal
    residuals and to assess the validity of that assumption.
-   Problem 2 asks you to model the time series of hourly
    weather-related variability at a tide gauge using an autoregressive
    model.
-   Problem 3 asks you to use Poisson regression to predict salamander
    counts based on environmental data.
-   Problem 4 (only required for students in BEE 5850) asks you to look
    at the impact of the gender of hurricane names on deaths[1].

[1] Yes, seriously. Ish. Trust me, I know.

### Load Environment

The following code loads the environment and makes sure all needed
packages are installed. This should be at the start of most Julia
scripts.

In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

The following packages are included in the environment (to help you find
other similar packages in other languages). The code below loads these
packages for use in the subsequent notebook (the desired functionality
for each package is commented next to the package).

In [1]:
using Random # random number generation and seed-setting
using DataFrames # tabular data structure
using CSV # reads/writes .csv files
using Distributions # interface to work with probability distributions
using Plots # plotting library
using StatsBase # statistical quantities like mean, median, etc
using StatsPlots # some additional statistical plotting tools
using Optim # optimization tools

## Problems

### Scoring

-   Problem 1 is worth 7 points.
-   Problem 2 is worth 6 points.
-   Problem 3 is worth 7 points.
-   Problem 4 is worth 5 points.

### Problem 1

First, load the data and normalize. We’ll plot the resulting series
(though this wasn’t required).

In [1]:
norm_yrs = 1980:1999

sl_dat = DataFrame(CSV.File(joinpath("data",  "CSIRO_Recons_gmsl_yr_2015.csv")))

rename!(sl_dat, [:Year, :GMSLR, :SD]) # rename to make columns easier to work with
sl_dat[!, :Year] .-= 0.5 # shift year to line up with years instead of being half-year 
sl_dat[!, :GMSLR] .-= mean(filter(row -> row.Year ∈ norm_yrs, sl_dat)[!, :GMSLR]) # rescale to be relative to 1880-1900 mean for consistency with temperature anomaly

# load temperature data
temp_dat = DataFrame(CSV.File(joinpath("data", "HadCRUT.5.0.2.0.analysis.summary_series.global.annual.csv")))
rename!(temp_dat, [:Year, :Temp, :Lower, :Upper]) # rename to make columns easier to work with
filter!(row -> row.Year ∈ sl_dat[!, :Year], temp_dat) # reduce to the same years that we have SL data for
temp_normalize = mean(filter(row -> row.Year ∈ norm_yrs, temp_dat)[!, :Temp]) # get renormalization to rescale temperature to 1880-1900 mean
temp_dat[!, :Temp] .-= temp_normalize
temp_dat[!, :Lower] .-= temp_normalize
temp_dat[!, :Upper] .-=  temp_normalize

sl_plot = scatter(sl_dat[!, :Year], sl_dat[!, :GMSLR], yerr=sl_dat[!, :SD], color=:black, label="Observations", ylabel="(mm)", xlabel="Year", title="Sea Level Anomaly")

temp_plot = scatter(temp_dat[!, :Year], temp_dat[!, :Temp], yerr=(temp_dat[!, :Temp] - temp_dat[!, :Lower], temp_dat[!, :Upper] - temp_dat[!, :Temp]), color=:black, label="Observations", ylabel="(°C)", xlabel="Year", title="Temperature")

plot(sl_plot, temp_plot, layout=(2, 1))

The discretized Grinsted (2010) model is:

$$
\begin{aligned}
S(t+1) &= S(t) + \delta t \frac{S_\text{eq}(t) - S(t)}{\tau} \\
S\text{eq}(t) &= a T(t) + b,
\end{aligned}
$$

where we take $\delta t = 1$. Then in Julia (as an example), this might
look like the following function.

In [1]:
function grinsted_slr(params, temps; Δt=1)
    a, b, τ, S₀ = params
    S = zeros(length(temps)) # initialize storage
    Seq = a * temps .+ b
    S[1] = S₀
    for i = 2:length(S)
        S[i] = S[i-1] +  Δt * (Seq[i] - S[i-1]) / τ
    end
    return S[1:end]
end

grinsted_slr (generic function with 1 method)

The next function lets us calculate the log-likelihood if we assume
i.i.d. Gaussian residuals. Note the inclusion of the observation errors

In [1]:
function gaussian_iid_loglik(params, temp_dat, slr_obs, slr_err, Δt=1.0)
    a, b, τ, S₀, σ = params 
    slr_sim = grinsted_slr((a, b, τ, S₀), temp_dat; Δt = Δt)
    ll = sum(logpdf.(Normal.(slr_sim, sqrt.(σ^2 .+ slr_err.^2)), slr_obs))
    return ll
end

gaussian_iid_loglik (generic function with 2 methods)

Now, let’s find the MLE by optimizing the `gaussian_iid_loglik`
function.

In [1]:
low_bds = [100.0, 100.0, 0.1, sl_dat.GMSLR[1] - 1.96 * sl_dat.SD[1], 0.0]
up_bds = [2000.0, 500.0, 2500.0, sl_dat.GMSLR[1] + 1.96 * sl_dat.SD[1], 100.0]
p₀ = [1000.0, 200.0, 1500.0, sl_dat.GMSLR[1], 10.0]

mle_optim = optimize(p -> -gaussian_iid_loglik(p, temp_dat.Temp, sl_dat.GMSLR, sl_dat.SD), low_bds, up_bds, p₀)
p_mle = mle_optim.minimizer
@show p_mle;

p_mle = [501.455739533761, 351.16508581722974, 153.6765067379414, -157.0640114587853, 1.2941470784564817e-9]

Notice that the inferred (uncertain) model error standard deviation is
zero; due to our statistical inability to distinguish between the two
Gaussian terms (model discrepancy and observation error) in the
residuals, the relatively large observation error subsumes any model
discrepancy term.

The sensitivity of GMSLR to increases in GMT is $a / \tau$:

$$S(t+1) = S(t) + \frac{\overbrace{(aT + b)}^{S_\text{eq}} - S(t)}{\tau}.$$

This value is 1.4 mm/(yr$\cdot ^\circ$C).

Finally, the i.i.d. Gaussian model had two key assumptions:
Gaussian-distributed residuals and independent residuals. To check the
first, we can use a Gaussian Q-Q plot, and the second, a partial
autocorrelation plot.

``` julia
grinsted_proj_mle = grinsted_slr(p_mle, temp_dat.Temp)
resids = sl_dat.GMSLR - grinsted_proj_mle
p1 = qqnorm(resids, ylabel="Theoretical Values", xlabel="Empirical Values")
pacf_resids = pacf(resids, 1:5)
p2 = plot(1:5, pacf_resids, marker=:circle, line=:stem, markersize=5, legend=false, ylabel="Partial Autocorrelation", xlabel="Time Lag")

display(p1)
display(p2)
```

<table>
<colgroup>
<col style="width: 50%" />
<col style="width: 50%" />
</colgroup>
<tbody>
<tr>
<td style="text-align: left;"><div width="50.0%"
data-layout-align="left">
<div>
<?xml version="1.0" encoding="utf-8"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="672" height="480" viewBox="0 0 2688 1920">
<defs>
  <clipPath id="clip330">
    <rect x="0" y="0" width="2688" height="1920"/>
  </clipPath>
</defs>
<path clip-path="url(#clip330)" d="M0 1920 L2688 1920 L2688 -4.26326e-14 L0 -4.26326e-14  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip331">
    <rect x="537" y="0" width="1883" height="1883"/>
  </clipPath>
</defs>
<path clip-path="url(#clip330)" d="M248.409 1734.12 L2640.76 1734.12 L2640.76 47.2441 L248.409 47.2441  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip332">
    <rect x="248" y="47" width="2393" height="1688"/>
  </clipPath>
</defs>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="378.33,1734.12 378.33,47.2441 "/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="740.266,1734.12 740.266,47.2441 "/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1102.2,1734.12 1102.2,47.2441 "/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1464.14,1734.12 1464.14,47.2441 "/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1826.07,1734.12 1826.07,47.2441 "/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="2188.01,1734.12 2188.01,47.2441 "/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="2549.95,1734.12 2549.95,47.2441 "/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="248.409,1642.51 2640.76,1642.51 "/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="248.409,1387.3 2640.76,1387.3 "/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="248.409,1132.1 2640.76,1132.1 "/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="248.409,876.892 2640.76,876.892 "/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="248.409,621.686 2640.76,621.686 "/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="248.409,366.481 2640.76,366.481 "/>
<polyline clip-path="url(#clip332)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="248.409,111.276 2640.76,111.276 "/>
<polyline clip-path="url(#clip330)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="248.409,1734.12 2640.76,1734.12 "/>
<polyline clip-path="url(#clip330)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="378.33,1734.12 378.33,1715.22 "/>
<polyline clip-path="url(#clip330)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="740.266,1734.12 740.266,1715.22 "/>
<polyline clip-path="url(#clip330)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1102.2,1734.12 1102.2,1715.22 "/>
<polyline clip-path="url(#clip330)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1464.14,1734.12 1464.14,1715.22 "/>
<polyline clip-path="url(#clip330)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1826.07,1734.12 1826.07,1715.22 "/>
<polyline clip-path="url(#clip330)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2188.01,1734.12 2188.01,1715.22 "/>
<polyline clip-path="url(#clip330)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2549.95,1734.12 2549.95,1715.22 "/>
<path clip-path="url(#clip330)" d="M332.89 1781.99 L362.566 1781.99 L362.566 1785.93 L332.89 1785.93 L332.89 1781.99 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M373.469 1794.89 L381.108 1794.89 L381.108 1768.52 L372.798 1770.19 L372.798 1765.93 L381.061 1764.26 L385.737 1764.26 L385.737 1794.89 L393.376 1794.89 L393.376 1798.82 L373.469 1798.82 L373.469 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M402.867 1764.26 L421.223 1764.26 L421.223 1768.2 L407.149 1768.2 L407.149 1776.67 Q408.168 1776.32 409.186 1776.16 Q410.205 1775.97 411.223 1775.97 Q417.01 1775.97 420.39 1779.15 Q423.77 1782.32 423.77 1787.73 Q423.77 1793.31 420.297 1796.41 Q416.825 1799.49 410.506 1799.49 Q408.33 1799.49 406.061 1799.12 Q403.816 1798.75 401.409 1798.01 L401.409 1793.31 Q403.492 1794.45 405.714 1795 Q407.936 1795.56 410.413 1795.56 Q414.418 1795.56 416.756 1793.45 Q419.094 1791.34 419.094 1787.73 Q419.094 1784.12 416.756 1782.02 Q414.418 1779.91 410.413 1779.91 Q408.538 1779.91 406.663 1780.33 Q404.811 1780.74 402.867 1781.62 L402.867 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M694.329 1781.99 L724.004 1781.99 L724.004 1785.93 L694.329 1785.93 L694.329 1781.99 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M734.907 1794.89 L742.546 1794.89 L742.546 1768.52 L734.236 1770.19 L734.236 1765.93 L742.5 1764.26 L747.176 1764.26 L747.176 1794.89 L754.814 1794.89 L754.814 1798.82 L734.907 1798.82 L734.907 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M774.259 1767.34 Q770.648 1767.34 768.819 1770.9 Q767.013 1774.45 767.013 1781.58 Q767.013 1788.68 768.819 1792.25 Q770.648 1795.79 774.259 1795.79 Q777.893 1795.79 779.699 1792.25 Q781.527 1788.68 781.527 1781.58 Q781.527 1774.45 779.699 1770.9 Q777.893 1767.34 774.259 1767.34 M774.259 1763.64 Q780.069 1763.64 783.124 1768.24 Q786.203 1772.83 786.203 1781.58 Q786.203 1790.3 783.124 1794.91 Q780.069 1799.49 774.259 1799.49 Q768.449 1799.49 765.37 1794.91 Q762.314 1790.3 762.314 1781.58 Q762.314 1772.83 765.37 1768.24 Q768.449 1763.64 774.259 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M1071.84 1781.99 L1101.52 1781.99 L1101.52 1785.93 L1071.84 1785.93 L1071.84 1781.99 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M1111.66 1764.26 L1130.01 1764.26 L1130.01 1768.2 L1115.94 1768.2 L1115.94 1776.67 Q1116.96 1776.32 1117.98 1776.16 Q1119 1775.97 1120.01 1775.97 Q1125.8 1775.97 1129.18 1779.15 Q1132.56 1782.32 1132.56 1787.73 Q1132.56 1793.31 1129.09 1796.41 Q1125.62 1799.49 1119.3 1799.49 Q1117.12 1799.49 1114.85 1799.12 Q1112.61 1798.75 1110.2 1798.01 L1110.2 1793.31 Q1112.28 1794.45 1114.5 1795 Q1116.73 1795.56 1119.2 1795.56 Q1123.21 1795.56 1125.55 1793.45 Q1127.88 1791.34 1127.88 1787.73 Q1127.88 1784.12 1125.55 1782.02 Q1123.21 1779.91 1119.2 1779.91 Q1117.33 1779.91 1115.45 1780.33 Q1113.6 1780.74 1111.66 1781.62 L1111.66 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M1464.14 1767.34 Q1460.53 1767.34 1458.7 1770.9 Q1456.89 1774.45 1456.89 1781.58 Q1456.89 1788.68 1458.7 1792.25 Q1460.53 1795.79 1464.14 1795.79 Q1467.77 1795.79 1469.58 1792.25 Q1471.41 1788.68 1471.41 1781.58 Q1471.41 1774.45 1469.58 1770.9 Q1467.77 1767.34 1464.14 1767.34 M1464.14 1763.64 Q1469.95 1763.64 1473 1768.24 Q1476.08 1772.83 1476.08 1781.58 Q1476.08 1790.3 1473 1794.91 Q1469.95 1799.49 1464.14 1799.49 Q1458.33 1799.49 1455.25 1794.91 Q1452.19 1790.3 1452.19 1781.58 Q1452.19 1772.83 1455.25 1768.24 Q1458.33 1763.64 1464.14 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M1816.35 1764.26 L1834.71 1764.26 L1834.71 1768.2 L1820.63 1768.2 L1820.63 1776.67 Q1821.65 1776.32 1822.67 1776.16 Q1823.69 1775.97 1824.71 1775.97 Q1830.49 1775.97 1833.87 1779.15 Q1837.25 1782.32 1837.25 1787.73 Q1837.25 1793.31 1833.78 1796.41 Q1830.31 1799.49 1823.99 1799.49 Q1821.81 1799.49 1819.55 1799.12 Q1817.3 1798.75 1814.89 1798.01 L1814.89 1793.31 Q1816.98 1794.45 1819.2 1795 Q1821.42 1795.56 1823.9 1795.56 Q1827.9 1795.56 1830.24 1793.45 Q1832.58 1791.34 1832.58 1787.73 Q1832.58 1784.12 1830.24 1782.02 Q1827.9 1779.91 1823.9 1779.91 Q1822.02 1779.91 1820.15 1780.33 Q1818.3 1780.74 1816.35 1781.62 L1816.35 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M2162.7 1794.89 L2170.34 1794.89 L2170.34 1768.52 L2162.03 1770.19 L2162.03 1765.93 L2170.29 1764.26 L2174.97 1764.26 L2174.97 1794.89 L2182.6 1794.89 L2182.6 1798.82 L2162.7 1798.82 L2162.7 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M2202.05 1767.34 Q2198.44 1767.34 2196.61 1770.9 Q2194.8 1774.45 2194.8 1781.58 Q2194.8 1788.68 2196.61 1792.25 Q2198.44 1795.79 2202.05 1795.79 Q2205.68 1795.79 2207.49 1792.25 Q2209.32 1788.68 2209.32 1781.58 Q2209.32 1774.45 2207.49 1770.9 Q2205.68 1767.34 2202.05 1767.34 M2202.05 1763.64 Q2207.86 1763.64 2210.91 1768.24 Q2213.99 1772.83 2213.99 1781.58 Q2213.99 1790.3 2210.91 1794.91 Q2207.86 1799.49 2202.05 1799.49 Q2196.24 1799.49 2193.16 1794.91 Q2190.1 1790.3 2190.1 1781.58 Q2190.1 1772.83 2193.16 1768.24 Q2196.24 1763.64 2202.05 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M2525.13 1794.89 L2532.77 1794.89 L2532.77 1768.52 L2524.46 1770.19 L2524.46 1765.93 L2532.72 1764.26 L2537.4 1764.26 L2537.4 1794.89 L2545.04 1794.89 L2545.04 1798.82 L2525.13 1798.82 L2525.13 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M2554.53 1764.26 L2572.89 1764.26 L2572.89 1768.2 L2558.81 1768.2 L2558.81 1776.67 Q2559.83 1776.32 2560.85 1776.16 Q2561.87 1775.97 2562.89 1775.97 Q2568.67 1775.97 2572.05 1779.15 Q2575.43 1782.32 2575.43 1787.73 Q2575.43 1793.31 2571.96 1796.41 Q2568.49 1799.49 2562.17 1799.49 Q2559.99 1799.49 2557.72 1799.12 Q2555.48 1798.75 2553.07 1798.01 L2553.07 1793.31 Q2555.15 1794.45 2557.38 1795 Q2559.6 1795.56 2562.08 1795.56 Q2566.08 1795.56 2568.42 1793.45 Q2570.76 1791.34 2570.76 1787.73 Q2570.76 1784.12 2568.42 1782.02 Q2566.08 1779.91 2562.08 1779.91 Q2560.2 1779.91 2558.33 1780.33 Q2556.47 1780.74 2554.53 1781.62 L2554.53 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M1181.04 1835.78 L1211.09 1835.78 L1211.09 1841.19 L1187.47 1841.19 L1187.47 1855.26 L1210.1 1855.26 L1210.1 1860.67 L1187.47 1860.67 L1187.47 1877.89 L1211.66 1877.89 L1211.66 1883.3 L1181.04 1883.3 L1181.04 1835.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M1249.73 1854.5 Q1251.92 1850.55 1254.98 1848.67 Q1258.04 1846.79 1262.17 1846.79 Q1267.74 1846.79 1270.77 1850.71 Q1273.79 1854.59 1273.79 1861.78 L1273.79 1883.3 L1267.9 1883.3 L1267.9 1861.98 Q1267.9 1856.85 1266.09 1854.37 Q1264.27 1851.89 1260.55 1851.89 Q1256 1851.89 1253.36 1854.91 Q1250.71 1857.93 1250.71 1863.15 L1250.71 1883.3 L1244.83 1883.3 L1244.83 1861.98 Q1244.83 1856.82 1243.01 1854.37 Q1241.2 1851.89 1237.41 1851.89 Q1232.92 1851.89 1230.28 1854.94 Q1227.64 1857.97 1227.64 1863.15 L1227.64 1883.3 L1221.75 1883.3 L1221.75 1847.65 L1227.64 1847.65 L1227.64 1853.19 Q1229.64 1849.91 1232.45 1848.35 Q1235.25 1846.79 1239.1 1846.79 Q1242.98 1846.79 1245.69 1848.77 Q1248.42 1850.74 1249.73 1854.5 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M1291.14 1877.95 L1291.14 1896.86 L1285.25 1896.86 L1285.25 1847.65 L1291.14 1847.65 L1291.14 1853.06 Q1292.98 1849.88 1295.78 1848.35 Q1298.62 1846.79 1302.53 1846.79 Q1309.02 1846.79 1313.07 1851.95 Q1317.14 1857.11 1317.14 1865.51 Q1317.14 1873.91 1313.07 1879.07 Q1309.02 1884.22 1302.53 1884.22 Q1298.62 1884.22 1295.78 1882.7 Q1292.98 1881.14 1291.14 1877.95 M1311.06 1865.51 Q1311.06 1859.05 1308.39 1855.39 Q1305.75 1851.7 1301.1 1851.7 Q1296.45 1851.7 1293.78 1855.39 Q1291.14 1859.05 1291.14 1865.51 Q1291.14 1871.97 1293.78 1875.66 Q1296.45 1879.32 1301.1 1879.32 Q1305.75 1879.32 1308.39 1875.66 Q1311.06 1871.97 1311.06 1865.51 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M1326.85 1847.65 L1332.71 1847.65 L1332.71 1883.3 L1326.85 1883.3 L1326.85 1847.65 M1326.85 1833.78 L1332.71 1833.78 L1332.71 1841.19 L1326.85 1841.19 L1326.85 1833.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M1365.62 1853.13 Q1364.63 1852.55 1363.45 1852.3 Q1362.31 1852.01 1360.91 1852.01 Q1355.94 1852.01 1353.27 1855.26 Q1350.62 1858.47 1350.62 1864.52 L1350.62 1883.3 L1344.74 1883.3 L1344.74 1847.65 L1350.62 1847.65 L1350.62 1853.19 Q1352.47 1849.94 1355.43 1848.39 Q1358.39 1846.79 1362.62 1846.79 Q1363.23 1846.79 1363.96 1846.89 Q1364.69 1846.95 1365.58 1847.11 L1365.62 1853.13 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M1371.76 1847.65 L1377.62 1847.65 L1377.62 1883.3 L1371.76 1883.3 L1371.76 1847.65 M1371.76 1833.78 L1377.62 1833.78 L1377.62 1841.19 L1371.76 1841.19 L1371.76 1833.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M1415.52 1849.02 L1415.52 1854.5 Q1413.04 1853.13 1410.53 1852.46 Q1408.04 1851.76 1405.5 1851.76 Q1399.8 1851.76 1396.65 1855.39 Q1393.5 1858.98 1393.5 1865.51 Q1393.5 1872.03 1396.65 1875.66 Q1399.8 1879.26 1405.5 1879.26 Q1408.04 1879.26 1410.53 1878.59 Q1413.04 1877.89 1415.52 1876.52 L1415.52 1881.93 Q1413.07 1883.08 1410.43 1883.65 Q1407.82 1884.22 1404.86 1884.22 Q1396.81 1884.22 1392.07 1879.16 Q1387.32 1874.1 1387.32 1865.51 Q1387.32 1856.79 1392.1 1851.79 Q1396.9 1846.79 1405.24 1846.79 Q1407.95 1846.79 1410.53 1847.37 Q1413.1 1847.91 1415.52 1849.02 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M1441.91 1865.38 Q1434.81 1865.38 1432.07 1867 Q1429.34 1868.63 1429.34 1872.54 Q1429.34 1875.66 1431.37 1877.51 Q1433.44 1879.32 1436.98 1879.32 Q1441.85 1879.32 1444.77 1875.88 Q1447.73 1872.42 1447.73 1866.69 L1447.73 1865.38 L1441.91 1865.38 M1453.59 1862.96 L1453.59 1883.3 L1447.73 1883.3 L1447.73 1877.89 Q1445.73 1881.14 1442.74 1882.7 Q1439.74 1884.22 1435.42 1884.22 Q1429.94 1884.22 1426.69 1881.17 Q1423.48 1878.08 1423.48 1872.92 Q1423.48 1866.91 1427.49 1863.85 Q1431.53 1860.8 1439.52 1860.8 L1447.73 1860.8 L1447.73 1860.23 Q1447.73 1856.18 1445.06 1853.99 Q1442.42 1851.76 1437.61 1851.76 Q1434.56 1851.76 1431.66 1852.49 Q1428.76 1853.22 1426.09 1854.69 L1426.09 1849.28 Q1429.3 1848.03 1432.33 1847.43 Q1435.35 1846.79 1438.22 1846.79 Q1445.95 1846.79 1449.77 1850.8 Q1453.59 1854.81 1453.59 1862.96 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M1465.65 1833.78 L1471.51 1833.78 L1471.51 1883.3 L1465.65 1883.3 L1465.65 1833.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M1516.99 1883.3 L1498.85 1835.78 L1505.57 1835.78 L1520.62 1875.79 L1535.71 1835.78 L1542.39 1835.78 L1524.28 1883.3 L1516.99 1883.3 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M1560.22 1865.38 Q1553.12 1865.38 1550.38 1867 Q1547.64 1868.63 1547.64 1872.54 Q1547.64 1875.66 1549.68 1877.51 Q1551.75 1879.32 1555.28 1879.32 Q1560.15 1879.32 1563.08 1875.88 Q1566.04 1872.42 1566.04 1866.69 L1566.04 1865.38 L1560.22 1865.38 M1571.9 1862.96 L1571.9 1883.3 L1566.04 1883.3 L1566.04 1877.89 Q1564.03 1881.14 1561.04 1882.7 Q1558.05 1884.22 1553.72 1884.22 Q1548.25 1884.22 1545 1881.17 Q1541.79 1878.08 1541.79 1872.92 Q1541.79 1866.91 1545.8 1863.85 Q1549.84 1860.8 1557.83 1860.8 L1566.04 1860.8 L1566.04 1860.23 Q1566.04 1856.18 1563.37 1853.99 Q1560.72 1851.76 1555.92 1851.76 Q1552.86 1851.76 1549.97 1852.49 Q1547.07 1853.22 1544.4 1854.69 L1544.4 1849.28 Q1547.61 1848.03 1550.64 1847.43 Q1553.66 1846.79 1556.52 1846.79 Q1564.26 1846.79 1568.08 1850.8 Q1571.9 1854.81 1571.9 1862.96 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M1583.96 1833.78 L1589.82 1833.78 L1589.82 1883.3 L1583.96 1883.3 L1583.96 1833.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M1601.47 1869.23 L1601.47 1847.65 L1607.32 1847.65 L1607.32 1869.01 Q1607.32 1874.07 1609.3 1876.62 Q1611.27 1879.13 1615.22 1879.13 Q1619.96 1879.13 1622.69 1876.11 Q1625.46 1873.08 1625.46 1867.86 L1625.46 1847.65 L1631.32 1847.65 L1631.32 1883.3 L1625.46 1883.3 L1625.46 1877.83 Q1623.33 1881.07 1620.5 1882.66 Q1617.7 1884.22 1613.97 1884.22 Q1607.83 1884.22 1604.65 1880.4 Q1601.47 1876.59 1601.47 1869.23 M1616.2 1846.79 L1616.2 1846.79 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M1673.88 1864.01 L1673.88 1866.88 L1646.95 1866.88 Q1647.33 1872.92 1650.58 1876.11 Q1653.86 1879.26 1659.68 1879.26 Q1663.05 1879.26 1666.2 1878.43 Q1669.39 1877.6 1672.51 1875.95 L1672.51 1881.49 Q1669.36 1882.82 1666.05 1883.52 Q1662.74 1884.22 1659.33 1884.22 Q1650.8 1884.22 1645.8 1879.26 Q1640.84 1874.29 1640.84 1865.83 Q1640.84 1857.07 1645.55 1851.95 Q1650.29 1846.79 1658.31 1846.79 Q1665.5 1846.79 1669.67 1851.44 Q1673.88 1856.06 1673.88 1864.01 M1668.02 1862.29 Q1667.96 1857.49 1665.31 1854.62 Q1662.7 1851.76 1658.37 1851.76 Q1653.47 1851.76 1650.51 1854.53 Q1647.58 1857.3 1647.14 1862.33 L1668.02 1862.29 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M1706.21 1848.7 L1706.21 1854.24 Q1703.73 1852.97 1701.06 1852.33 Q1698.38 1851.7 1695.52 1851.7 Q1691.16 1851.7 1688.96 1853.03 Q1686.8 1854.37 1686.8 1857.04 Q1686.8 1859.08 1688.36 1860.26 Q1689.92 1861.4 1694.63 1862.45 L1696.63 1862.9 Q1702.87 1864.24 1705.48 1866.69 Q1708.12 1869.11 1708.12 1873.47 Q1708.12 1878.43 1704.18 1881.33 Q1700.26 1884.22 1693.39 1884.22 Q1690.52 1884.22 1687.4 1883.65 Q1684.31 1883.11 1680.88 1882 L1680.88 1875.95 Q1684.12 1877.64 1687.27 1878.49 Q1690.43 1879.32 1693.51 1879.32 Q1697.65 1879.32 1699.88 1877.92 Q1702.11 1876.49 1702.11 1873.91 Q1702.11 1871.52 1700.48 1870.25 Q1698.89 1868.98 1693.45 1867.8 L1691.41 1867.32 Q1685.97 1866.18 1683.55 1863.82 Q1681.13 1861.43 1681.13 1857.3 Q1681.13 1852.27 1684.7 1849.53 Q1688.26 1846.79 1694.82 1846.79 Q1698.06 1846.79 1700.93 1847.27 Q1703.79 1847.75 1706.21 1848.7 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip330)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="248.409,1734.12 248.409,47.2441 "/>
<polyline clip-path="url(#clip330)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="248.409,1642.51 267.307,1642.51 "/>
<polyline clip-path="url(#clip330)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="248.409,1387.3 267.307,1387.3 "/>
<polyline clip-path="url(#clip330)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="248.409,1132.1 267.307,1132.1 "/>
<polyline clip-path="url(#clip330)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="248.409,876.892 267.307,876.892 "/>
<polyline clip-path="url(#clip330)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="248.409,621.686 267.307,621.686 "/>
<polyline clip-path="url(#clip330)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="248.409,366.481 267.307,366.481 "/>
<polyline clip-path="url(#clip330)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="248.409,111.276 267.307,111.276 "/>
<path clip-path="url(#clip330)" d="M117.21 1642.96 L146.886 1642.96 L146.886 1646.89 L117.21 1646.89 L117.21 1642.96 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M157.788 1655.85 L165.427 1655.85 L165.427 1629.49 L157.117 1631.15 L157.117 1626.89 L165.381 1625.23 L170.057 1625.23 L170.057 1655.85 L177.696 1655.85 L177.696 1659.79 L157.788 1659.79 L157.788 1655.85 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M187.186 1625.23 L205.543 1625.23 L205.543 1629.16 L191.469 1629.16 L191.469 1637.64 Q192.487 1637.29 193.506 1637.13 Q194.524 1636.94 195.543 1636.94 Q201.33 1636.94 204.709 1640.11 Q208.089 1643.28 208.089 1648.7 Q208.089 1654.28 204.617 1657.38 Q201.145 1660.46 194.825 1660.46 Q192.649 1660.46 190.381 1660.09 Q188.135 1659.72 185.728 1658.98 L185.728 1654.28 Q187.811 1655.41 190.033 1655.97 Q192.256 1656.52 194.733 1656.52 Q198.737 1656.52 201.075 1654.42 Q203.413 1652.31 203.413 1648.7 Q203.413 1645.09 201.075 1642.98 Q198.737 1640.88 194.733 1640.88 Q192.858 1640.88 190.983 1641.29 Q189.131 1641.71 187.186 1642.59 L187.186 1625.23 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M116.214 1387.75 L145.89 1387.75 L145.89 1391.69 L116.214 1391.69 L116.214 1387.75 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M156.793 1400.65 L164.432 1400.65 L164.432 1374.28 L156.122 1375.95 L156.122 1371.69 L164.385 1370.02 L169.061 1370.02 L169.061 1400.65 L176.7 1400.65 L176.7 1404.58 L156.793 1404.58 L156.793 1400.65 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M196.145 1373.1 Q192.533 1373.1 190.705 1376.67 Q188.899 1380.21 188.899 1387.34 Q188.899 1394.44 190.705 1398.01 Q192.533 1401.55 196.145 1401.55 Q199.779 1401.55 201.584 1398.01 Q203.413 1394.44 203.413 1387.34 Q203.413 1380.21 201.584 1376.67 Q199.779 1373.1 196.145 1373.1 M196.145 1369.4 Q201.955 1369.4 205.01 1374 Q208.089 1378.59 208.089 1387.34 Q208.089 1396.06 205.01 1400.67 Q201.955 1405.25 196.145 1405.25 Q190.334 1405.25 187.256 1400.67 Q184.2 1396.06 184.2 1387.34 Q184.2 1378.59 187.256 1374 Q190.334 1369.4 196.145 1369.4 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M147.372 1132.55 L177.047 1132.55 L177.047 1136.48 L147.372 1136.48 L147.372 1132.55 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M187.186 1114.82 L205.543 1114.82 L205.543 1118.75 L191.469 1118.75 L191.469 1127.22 Q192.487 1126.88 193.506 1126.72 Q194.524 1126.53 195.543 1126.53 Q201.33 1126.53 204.709 1129.7 Q208.089 1132.87 208.089 1138.29 Q208.089 1143.87 204.617 1146.97 Q201.145 1150.05 194.825 1150.05 Q192.649 1150.05 190.381 1149.68 Q188.135 1149.31 185.728 1148.57 L185.728 1143.87 Q187.811 1145 190.033 1145.56 Q192.256 1146.11 194.733 1146.11 Q198.737 1146.11 201.075 1144.01 Q203.413 1141.9 203.413 1138.29 Q203.413 1134.68 201.075 1132.57 Q198.737 1130.47 194.733 1130.47 Q192.858 1130.47 190.983 1130.88 Q189.131 1131.3 187.186 1132.18 L187.186 1114.82 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M196.145 862.691 Q192.533 862.691 190.705 866.255 Q188.899 869.797 188.899 876.927 Q188.899 884.033 190.705 887.598 Q192.533 891.139 196.145 891.139 Q199.779 891.139 201.584 887.598 Q203.413 884.033 203.413 876.927 Q203.413 869.797 201.584 866.255 Q199.779 862.691 196.145 862.691 M196.145 858.987 Q201.955 858.987 205.01 863.593 Q208.089 868.177 208.089 876.927 Q208.089 885.653 205.01 890.26 Q201.955 894.843 196.145 894.843 Q190.334 894.843 187.256 890.26 Q184.2 885.653 184.2 876.927 Q184.2 868.177 187.256 863.593 Q190.334 858.987 196.145 858.987 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M187.186 604.406 L205.543 604.406 L205.543 608.342 L191.469 608.342 L191.469 616.814 Q192.487 616.467 193.506 616.305 Q194.524 616.119 195.543 616.119 Q201.33 616.119 204.709 619.291 Q208.089 622.462 208.089 627.879 Q208.089 633.457 204.617 636.559 Q201.145 639.638 194.825 639.638 Q192.649 639.638 190.381 639.267 Q188.135 638.897 185.728 638.156 L185.728 633.457 Q187.811 634.591 190.033 635.147 Q192.256 635.703 194.733 635.703 Q198.737 635.703 201.075 633.596 Q203.413 631.49 203.413 627.879 Q203.413 624.267 201.075 622.161 Q198.737 620.055 194.733 620.055 Q192.858 620.055 190.983 620.471 Q189.131 620.888 187.186 621.767 L187.186 604.406 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M156.793 379.826 L164.432 379.826 L164.432 353.46 L156.122 355.127 L156.122 350.868 L164.385 349.201 L169.061 349.201 L169.061 379.826 L176.7 379.826 L176.7 383.761 L156.793 383.761 L156.793 379.826 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M196.145 352.28 Q192.533 352.28 190.705 355.844 Q188.899 359.386 188.899 366.516 Q188.899 373.622 190.705 377.187 Q192.533 380.729 196.145 380.729 Q199.779 380.729 201.584 377.187 Q203.413 373.622 203.413 366.516 Q203.413 359.386 201.584 355.844 Q199.779 352.28 196.145 352.28 M196.145 348.576 Q201.955 348.576 205.01 353.182 Q208.089 357.766 208.089 366.516 Q208.089 375.243 205.01 379.849 Q201.955 384.432 196.145 384.432 Q190.334 384.432 187.256 379.849 Q184.2 375.243 184.2 366.516 Q184.2 357.766 187.256 353.182 Q190.334 348.576 196.145 348.576 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M157.788 124.62 L165.427 124.62 L165.427 98.2548 L157.117 99.9215 L157.117 95.6623 L165.381 93.9956 L170.057 93.9956 L170.057 124.62 L177.696 124.62 L177.696 128.556 L157.788 128.556 L157.788 124.62 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M187.186 93.9956 L205.543 93.9956 L205.543 97.9308 L191.469 97.9308 L191.469 106.403 Q192.487 106.056 193.506 105.894 Q194.524 105.708 195.543 105.708 Q201.33 105.708 204.709 108.88 Q208.089 112.051 208.089 117.468 Q208.089 123.046 204.617 126.148 Q201.145 129.227 194.825 129.227 Q192.649 129.227 190.381 128.857 Q188.135 128.486 185.728 127.745 L185.728 123.046 Q187.811 124.181 190.033 124.736 Q192.256 125.292 194.733 125.292 Q198.737 125.292 201.075 123.185 Q203.413 121.079 203.413 117.468 Q203.413 113.857 201.075 111.75 Q198.737 109.644 194.733 109.644 Q192.858 109.644 190.983 110.06 Q189.131 110.477 187.186 111.357 L187.186 93.9956 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M14.1184 1188.1 L14.1184 1147.9 L19.5293 1147.9 L19.5293 1164.77 L61.6384 1164.77 L61.6384 1171.23 L19.5293 1171.23 L19.5293 1188.1 L14.1184 1188.1 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M40.1224 1112.32 L61.6384 1112.32 L61.6384 1118.17 L40.3133 1118.17 Q35.2526 1118.17 32.7381 1120.15 Q30.2237 1122.12 30.2237 1126.07 Q30.2237 1130.81 33.2474 1133.55 Q36.2711 1136.29 41.491 1136.29 L61.6384 1136.29 L61.6384 1142.17 L12.1132 1142.17 L12.1132 1136.29 L31.5287 1136.29 Q28.314 1134.18 26.7225 1131.35 Q25.1311 1128.49 25.1311 1124.76 Q25.1311 1118.62 28.9505 1115.47 Q32.7381 1112.32 40.1224 1112.32 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M42.3504 1070.15 L45.2149 1070.15 L45.2149 1097.07 Q51.2623 1096.69 54.4452 1093.44 Q57.5962 1090.17 57.5962 1084.34 Q57.5962 1080.97 56.7687 1077.82 Q55.9411 1074.63 54.2861 1071.51 L59.8242 1071.51 Q61.161 1074.67 61.8612 1077.98 Q62.5615 1081.29 62.5615 1084.69 Q62.5615 1093.22 57.5962 1098.22 Q52.631 1103.18 44.1646 1103.18 Q35.4117 1103.18 30.2873 1098.47 Q25.1311 1093.73 25.1311 1085.71 Q25.1311 1078.52 29.7781 1074.35 Q34.3932 1070.15 42.3504 1070.15 M40.6316 1076 Q35.8255 1076.07 32.9609 1078.71 Q30.0964 1081.32 30.0964 1085.65 Q30.0964 1090.55 32.8654 1093.51 Q35.6345 1096.44 40.6634 1096.88 L40.6316 1076 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M30.0964 1046.72 Q30.0964 1051.43 33.7885 1054.17 Q37.4488 1056.91 43.8463 1056.91 Q50.2438 1056.91 53.9359 1054.2 Q57.5962 1051.46 57.5962 1046.72 Q57.5962 1042.04 53.9041 1039.3 Q50.212 1036.57 43.8463 1036.57 Q37.5124 1036.57 33.8203 1039.3 Q30.0964 1042.04 30.0964 1046.72 M25.1311 1046.72 Q25.1311 1039.08 30.0964 1034.72 Q35.0616 1030.36 43.8463 1030.36 Q52.5991 1030.36 57.5962 1034.72 Q62.5615 1039.08 62.5615 1046.72 Q62.5615 1054.39 57.5962 1058.75 Q52.5991 1063.08 43.8463 1063.08 Q35.0616 1063.08 30.0964 1058.75 Q25.1311 1054.39 25.1311 1046.72 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M31.465 999.996 Q30.8921 1000.98 30.6375 1002.16 Q30.351 1003.31 30.351 1004.71 Q30.351 1009.67 33.5975 1012.35 Q36.8122 1014.99 42.8596 1014.99 L61.6384 1014.99 L61.6384 1020.88 L25.9905 1020.88 L25.9905 1014.99 L31.5287 1014.99 Q28.2821 1013.14 26.7225 1010.18 Q25.1311 1007.22 25.1311 1002.99 Q25.1311 1002.38 25.2266 1001.65 Q25.2903 1000.92 25.4494 1000.03 L31.465 999.996 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M42.3504 964.793 L45.2149 964.793 L45.2149 991.72 Q51.2623 991.338 54.4452 988.092 Q57.5962 984.813 57.5962 978.989 Q57.5962 975.615 56.7687 972.464 Q55.9411 969.281 54.2861 966.162 L59.8242 966.162 Q61.161 969.313 61.8612 972.623 Q62.5615 975.933 62.5615 979.339 Q62.5615 987.869 57.5962 992.866 Q52.631 997.831 44.1646 997.831 Q35.4117 997.831 30.2873 993.121 Q25.1311 988.378 25.1311 980.357 Q25.1311 973.164 29.7781 968.995 Q34.3932 964.793 42.3504 964.793 M40.6316 970.65 Q35.8255 970.713 32.9609 973.355 Q30.0964 975.965 30.0964 980.294 Q30.0964 985.195 32.8654 988.155 Q35.6345 991.084 40.6634 991.529 L40.6316 970.65 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M15.869 949.388 L25.9905 949.388 L25.9905 937.325 L30.542 937.325 L30.542 949.388 L49.8937 949.388 Q54.2542 949.388 55.4955 948.211 Q56.7368 947.001 56.7368 943.341 L56.7368 937.325 L61.6384 937.325 L61.6384 943.341 Q61.6384 950.12 59.124 952.698 Q56.5777 955.277 49.8937 955.277 L30.542 955.277 L30.542 959.573 L25.9905 959.573 L25.9905 955.277 L15.869 955.277 L15.869 949.388 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M25.9905 929.623 L25.9905 923.766 L61.6384 923.766 L61.6384 929.623 L25.9905 929.623 M12.1132 929.623 L12.1132 923.766 L19.5293 923.766 L19.5293 929.623 L12.1132 929.623 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M27.3591 885.859 L32.8336 885.859 Q31.465 888.341 30.7966 890.856 Q30.0964 893.338 30.0964 895.885 Q30.0964 901.582 33.7248 904.733 Q37.3214 907.884 43.8463 907.884 Q50.3711 907.884 53.9996 904.733 Q57.5962 901.582 57.5962 895.885 Q57.5962 893.338 56.9278 890.856 Q56.2276 888.341 54.859 885.859 L60.2698 885.859 Q61.4156 888.309 61.9886 890.951 Q62.5615 893.561 62.5615 896.521 Q62.5615 904.574 57.5007 909.316 Q52.44 914.059 43.8463 914.059 Q35.1253 914.059 30.1282 909.284 Q25.1311 904.478 25.1311 896.139 Q25.1311 893.434 25.704 890.856 Q26.2451 888.277 27.3591 885.859 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M43.719 859.473 Q43.719 866.57 45.3422 869.308 Q46.9655 872.045 50.8804 872.045 Q53.9996 872.045 55.8457 870.008 Q57.6599 867.939 57.6599 864.406 Q57.6599 859.536 54.2224 856.608 Q50.7531 853.648 45.0239 853.648 L43.719 853.648 L43.719 859.473 M41.3 847.792 L61.6384 847.792 L61.6384 853.648 L56.2276 853.648 Q59.4741 855.653 61.0337 858.645 Q62.5615 861.637 62.5615 865.966 Q62.5615 871.44 59.5059 874.687 Q56.4186 877.901 51.2623 877.901 Q45.2467 877.901 42.1912 873.891 Q39.1357 869.849 39.1357 861.86 L39.1357 853.648 L38.5628 853.648 Q34.5205 853.648 32.3244 856.322 Q30.0964 858.963 30.0964 863.77 Q30.0964 866.825 30.8284 869.721 Q31.5605 872.618 33.0246 875.291 L27.6137 875.291 Q26.3724 872.077 25.7677 869.053 Q25.1311 866.029 25.1311 863.165 Q25.1311 855.43 29.1415 851.611 Q33.1519 847.792 41.3 847.792 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M12.1132 835.729 L12.1132 829.872 L61.6384 829.872 L61.6384 835.729 L12.1132 835.729 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M61.6384 784.389 L14.1184 802.531 L14.1184 795.816 L54.1269 780.761 L14.1184 765.674 L14.1184 758.99 L61.6384 777.1 L61.6384 784.389 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M43.719 741.166 Q43.719 748.264 45.3422 751.001 Q46.9655 753.738 50.8804 753.738 Q53.9996 753.738 55.8457 751.701 Q57.6599 749.632 57.6599 746.099 Q57.6599 741.23 54.2224 738.301 Q50.7531 735.341 45.0239 735.341 L43.719 735.341 L43.719 741.166 M41.3 729.485 L61.6384 729.485 L61.6384 735.341 L56.2276 735.341 Q59.4741 737.347 61.0337 740.338 Q62.5615 743.33 62.5615 747.659 Q62.5615 753.134 59.5059 756.38 Q56.4186 759.595 51.2623 759.595 Q45.2467 759.595 42.1912 755.584 Q39.1357 751.542 39.1357 743.553 L39.1357 735.341 L38.5628 735.341 Q34.5205 735.341 32.3244 738.015 Q30.0964 740.657 30.0964 745.463 Q30.0964 748.518 30.8284 751.415 Q31.5605 754.311 33.0246 756.985 L27.6137 756.985 Q26.3724 753.77 25.7677 750.746 Q25.1311 747.723 25.1311 744.858 Q25.1311 737.124 29.1415 733.304 Q33.1519 729.485 41.3 729.485 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M12.1132 717.422 L12.1132 711.565 L61.6384 711.565 L61.6384 717.422 L12.1132 717.422 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M47.5702 699.916 L25.9905 699.916 L25.9905 694.06 L47.3474 694.06 Q52.4082 694.06 54.9545 692.086 Q57.4689 690.113 57.4689 686.166 Q57.4689 681.424 54.4452 678.687 Q51.4215 675.918 46.2016 675.918 L25.9905 675.918 L25.9905 670.061 L61.6384 670.061 L61.6384 675.918 L56.1639 675.918 Q59.4104 678.05 61.0019 680.883 Q62.5615 683.684 62.5615 687.408 Q62.5615 693.551 58.742 696.733 Q54.9226 699.916 47.5702 699.916 M25.1311 685.18 L25.1311 685.18 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M42.3504 627.506 L45.2149 627.506 L45.2149 654.433 Q51.2623 654.051 54.4452 650.805 Q57.5962 647.526 57.5962 641.702 Q57.5962 638.328 56.7687 635.177 Q55.9411 631.994 54.2861 628.875 L59.8242 628.875 Q61.161 632.026 61.8612 635.336 Q62.5615 638.646 62.5615 642.052 Q62.5615 650.582 57.5962 655.579 Q52.631 660.544 44.1646 660.544 Q35.4117 660.544 30.2873 655.834 Q25.1311 651.091 25.1311 643.07 Q25.1311 635.877 29.7781 631.708 Q34.3932 627.506 42.3504 627.506 M40.6316 633.363 Q35.8255 633.426 32.9609 636.068 Q30.0964 638.678 30.0964 643.007 Q30.0964 647.908 32.8654 650.868 Q35.6345 653.797 40.6634 654.242 L40.6316 633.363 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip330)" d="M27.0408 595.169 L32.579 595.169 Q31.3059 597.651 30.6693 600.325 Q30.0327 602.998 30.0327 605.863 Q30.0327 610.223 31.3695 612.42 Q32.7063 614.584 35.3799 614.584 Q37.4169 614.584 38.5946 613.024 Q39.7404 611.465 40.7908 606.754 L41.2364 604.749 Q42.5732 598.511 45.0239 595.901 Q47.4429 593.259 51.8034 593.259 Q56.7687 593.259 59.6651 597.206 Q62.5615 601.12 62.5615 607.995 Q62.5615 610.86 61.9886 613.979 Q61.4475 617.067 60.3335 620.504 L54.2861 620.504 Q55.973 617.258 56.8323 614.107 Q57.6599 610.955 57.6599 607.868 Q57.6599 603.73 56.2594 601.502 Q54.8271 599.274 52.249 599.274 Q49.8619 599.274 48.5887 600.898 Q47.3156 602.489 46.1379 607.932 L45.6605 609.969 Q44.5147 615.411 42.1594 617.83 Q39.7722 620.249 35.6345 620.249 Q30.6056 620.249 27.8684 616.685 Q25.1311 613.12 25.1311 606.563 Q25.1311 603.317 25.6085 600.452 Q26.086 597.587 27.0408 595.169 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip332)" style="stroke:#6b9e32; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="316.117,1686.38 2573.05,94.9858 "/>
<circle clip-path="url(#clip332)" cx="316.117" cy="1620.79" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="481.347" cy="1484.33" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="566.294" cy="1442.02" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="625.832" cy="1425.42" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="672.476" cy="1402.07" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="711.232" cy="1373.5" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="744.632" cy="1329.98" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="774.143" cy="1298.61" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="800.696" cy="1293.88" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="824.917" cy="1281.41" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="847.251" cy="1259.84" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="868.026" cy="1243.28" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="887.489" cy="1238.86" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="905.833" cy="1235.71" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="923.212" cy="1230.14" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="939.747" cy="1214.76" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="955.541" cy="1198.65" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="970.677" cy="1189.26" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="985.227" cy="1183.08" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="999.249" cy="1181.78" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1012.8" cy="1178.29" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1025.91" cy="1161.1" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1038.63" cy="1136.41" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1051" cy="1129.68" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1063.03" cy="1124.98" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1074.76" cy="1121.94" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1086.22" cy="1118.66" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1097.41" cy="1113.16" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1108.36" cy="1111.6" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1119.1" cy="1110.34" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1129.62" cy="1106.53" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1139.95" cy="1103.31" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1150.11" cy="1103.07" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1160.09" cy="1102.08" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1169.92" cy="1099.32" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1179.6" cy="1095.63" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1189.15" cy="1088.85" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1198.56" cy="1087.31" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1207.86" cy="1086.04" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1217.04" cy="1081.62" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1226.11" cy="1077.49" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1235.08" cy="1074" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1243.96" cy="1067.19" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1252.76" cy="1051.81" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1261.46" cy="1047.68" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1270.1" cy="1040.62" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1278.66" cy="1038.68" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1287.15" cy="1037.87" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1295.58" cy="1035.63" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1303.94" cy="1030.43" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1312.26" cy="1026.21" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1320.52" cy="1014.34" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1328.74" cy="1008.97" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1336.91" cy="994.121" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1345.04" cy="992.771" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1353.14" cy="980.559" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1361.2" cy="979.642" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1369.23" cy="979.262" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1377.23" cy="974.929" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1385.21" cy="963.288" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1393.17" cy="955.571" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1401.11" cy="946.604" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1409.04" cy="942.594" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1416.95" cy="939.526" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1424.85" cy="937.439" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1432.75" cy="928.373" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1440.64" cy="911.637" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1448.53" cy="911.266" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1456.42" cy="911.169" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1464.31" cy="910.418" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1472.22" cy="908.717" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1480.13" cy="907.549" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1488.05" cy="900.443" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1495.99" cy="890.209" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1503.95" cy="873.907" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1511.93" cy="867.348" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1519.94" cy="864.085" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1527.97" cy="851.905" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1536.03" cy="841.28" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1544.12" cy="840.146" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1552.26" cy="836.258" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1560.43" cy="809.244" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1568.64" cy="802.775" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1576.91" cy="802.326" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1585.22" cy="800.673" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1593.59" cy="795.564" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1602.02" cy="794.311" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1610.51" cy="788.994" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1619.07" cy="785.947" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1627.7" cy="784.113" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1636.41" cy="776.064" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1645.2" cy="766.126" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1654.08" cy="748.455" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1663.05" cy="743.846" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1672.13" cy="738.737" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1681.31" cy="725.333" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1690.6" cy="715.334" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1700.02" cy="706.592" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1709.56" cy="704.541" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1719.24" cy="695.764" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1729.07" cy="687.807" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1739.06" cy="678.099" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1749.21" cy="668.922" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1759.54" cy="660.015" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1770.07" cy="654.653" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1780.8" cy="648.335" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1791.76" cy="637.252" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1802.95" cy="633.051" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1814.4" cy="626.274" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1826.13" cy="623.102" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1838.17" cy="620.085" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1850.53" cy="614.553" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1863.25" cy="604.645" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1876.37" cy="599.041" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1889.92" cy="578.185" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1903.94" cy="563.719" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1918.49" cy="552.818" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1933.62" cy="534.406" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1949.42" cy="517.051" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1965.95" cy="509.359" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="1983.33" cy="504.74" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="2001.68" cy="499.975" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="2021.14" cy="467.241" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="2041.91" cy="440.365" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="2064.25" cy="429.258" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="2088.47" cy="408.247" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="2115.02" cy="376.867" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="2144.53" cy="358.016" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="2177.93" cy="319.212" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="2216.69" cy="279.147" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="2263.33" cy="255.577" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="2322.87" cy="227.392" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="2407.82" cy="210.098" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip332)" cx="2573.05" cy="166.516" r="14.4" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
</svg>
<p>(a) Q-Q Plot vs. a Theoretical Normal Distribution</p>
</div>
</div></td>
<td style="text-align: left;"><div width="50.0%"
data-layout-align="left">
<div>
<?xml version="1.0" encoding="utf-8"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="672" height="480" viewBox="0 0 2688 1920">
<defs>
  <clipPath id="clip390">
    <rect x="0" y="0" width="2688" height="1920"/>
  </clipPath>
</defs>
<path clip-path="url(#clip390)" d="M0 1920 L2688 1920 L2688 -4.26326e-14 L0 -4.26326e-14  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip391">
    <rect x="537" y="0" width="1883" height="1883"/>
  </clipPath>
</defs>
<path clip-path="url(#clip390)" d="M226.141 1734.12 L2640.76 1734.12 L2640.76 47.2441 L226.141 47.2441  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip392">
    <rect x="226" y="47" width="2416" height="1688"/>
  </clipPath>
</defs>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="294.479,1734.12 294.479,47.2441 "/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="863.963,1734.12 863.963,47.2441 "/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1433.45,1734.12 1433.45,47.2441 "/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="2002.93,1734.12 2002.93,47.2441 "/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="2572.42,1734.12 2572.42,47.2441 "/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="226.141,1497.09 2640.76,1497.09 "/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="226.141,1230.66 2640.76,1230.66 "/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="226.141,964.222 2640.76,964.222 "/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="226.141,697.785 2640.76,697.785 "/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="226.141,431.349 2640.76,431.349 "/>
<polyline clip-path="url(#clip392)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="226.141,164.913 2640.76,164.913 "/>
<polyline clip-path="url(#clip390)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.141,1734.12 2640.76,1734.12 "/>
<polyline clip-path="url(#clip390)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="294.479,1734.12 294.479,1715.22 "/>
<polyline clip-path="url(#clip390)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="863.963,1734.12 863.963,1715.22 "/>
<polyline clip-path="url(#clip390)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1433.45,1734.12 1433.45,1715.22 "/>
<polyline clip-path="url(#clip390)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2002.93,1734.12 2002.93,1715.22 "/>
<polyline clip-path="url(#clip390)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2572.42,1734.12 2572.42,1715.22 "/>
<path clip-path="url(#clip390)" d="M284.861 1794.89 L292.5 1794.89 L292.5 1768.52 L284.189 1770.19 L284.189 1765.93 L292.453 1764.26 L297.129 1764.26 L297.129 1794.89 L304.768 1794.89 L304.768 1798.82 L284.861 1798.82 L284.861 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M858.616 1794.89 L874.936 1794.89 L874.936 1798.82 L852.991 1798.82 L852.991 1794.89 Q855.653 1792.13 860.237 1787.5 Q864.843 1782.85 866.024 1781.51 Q868.269 1778.98 869.149 1777.25 Q870.051 1775.49 870.051 1773.8 Q870.051 1771.04 868.107 1769.31 Q866.186 1767.57 863.084 1767.57 Q860.885 1767.57 858.431 1768.34 Q856.001 1769.1 853.223 1770.65 L853.223 1765.93 Q856.047 1764.79 858.501 1764.21 Q860.954 1763.64 862.991 1763.64 Q868.362 1763.64 871.556 1766.32 Q874.75 1769.01 874.75 1773.5 Q874.75 1775.63 873.94 1777.55 Q873.153 1779.45 871.047 1782.04 Q870.468 1782.71 867.366 1785.93 Q864.264 1789.12 858.616 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M1437.7 1780.19 Q1441.05 1780.9 1442.93 1783.17 Q1444.83 1785.44 1444.83 1788.77 Q1444.83 1793.89 1441.31 1796.69 Q1437.79 1799.49 1431.31 1799.49 Q1429.13 1799.49 1426.82 1799.05 Q1424.52 1798.64 1422.07 1797.78 L1422.07 1793.27 Q1424.02 1794.4 1426.33 1794.98 Q1428.65 1795.56 1431.17 1795.56 Q1435.57 1795.56 1437.86 1793.82 Q1440.17 1792.08 1440.17 1788.77 Q1440.17 1785.72 1438.02 1784.01 Q1435.89 1782.27 1432.07 1782.27 L1428.04 1782.27 L1428.04 1778.43 L1432.26 1778.43 Q1435.71 1778.43 1437.53 1777.06 Q1439.36 1775.67 1439.36 1773.08 Q1439.36 1770.42 1437.46 1769.01 Q1435.59 1767.57 1432.07 1767.57 Q1430.15 1767.57 1427.95 1767.99 Q1425.75 1768.4 1423.11 1769.28 L1423.11 1765.12 Q1425.77 1764.38 1428.09 1764.01 Q1430.43 1763.64 1432.49 1763.64 Q1437.81 1763.64 1440.91 1766.07 Q1444.02 1768.47 1444.02 1772.59 Q1444.02 1775.46 1442.37 1777.46 Q1440.73 1779.42 1437.7 1780.19 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M2005.94 1768.34 L1994.14 1786.78 L2005.94 1786.78 L2005.94 1768.34 M2004.72 1764.26 L2010.59 1764.26 L2010.59 1786.78 L2015.53 1786.78 L2015.53 1790.67 L2010.59 1790.67 L2010.59 1798.82 L2005.94 1798.82 L2005.94 1790.67 L1990.34 1790.67 L1990.34 1786.16 L2004.72 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M2562.7 1764.26 L2581.05 1764.26 L2581.05 1768.2 L2566.98 1768.2 L2566.98 1776.67 Q2568 1776.32 2569.01 1776.16 Q2570.03 1775.97 2571.05 1775.97 Q2576.84 1775.97 2580.22 1779.15 Q2583.6 1782.32 2583.6 1787.73 Q2583.6 1793.31 2580.13 1796.41 Q2576.65 1799.49 2570.33 1799.49 Q2568.16 1799.49 2565.89 1799.12 Q2563.64 1798.75 2561.24 1798.01 L2561.24 1793.31 Q2563.32 1794.45 2565.54 1795 Q2567.76 1795.56 2570.24 1795.56 Q2574.25 1795.56 2576.58 1793.45 Q2578.92 1791.34 2578.92 1787.73 Q2578.92 1784.12 2576.58 1782.02 Q2574.25 1779.91 2570.24 1779.91 Q2568.37 1779.91 2566.49 1780.33 Q2564.64 1780.74 2562.7 1781.62 L2562.7 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M1287.37 1835.78 L1327.57 1835.78 L1327.57 1841.19 L1310.7 1841.19 L1310.7 1883.3 L1304.24 1883.3 L1304.24 1841.19 L1287.37 1841.19 L1287.37 1835.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M1331.52 1847.65 L1337.37 1847.65 L1337.37 1883.3 L1331.52 1883.3 L1331.52 1847.65 M1331.52 1833.78 L1337.37 1833.78 L1337.37 1841.19 L1331.52 1841.19 L1331.52 1833.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M1377.38 1854.5 Q1379.58 1850.55 1382.63 1848.67 Q1385.69 1846.79 1389.83 1846.79 Q1395.4 1846.79 1398.42 1850.71 Q1401.44 1854.59 1401.44 1861.78 L1401.44 1883.3 L1395.56 1883.3 L1395.56 1861.98 Q1395.56 1856.85 1393.74 1854.37 Q1391.93 1851.89 1388.2 1851.89 Q1383.65 1851.89 1381.01 1854.91 Q1378.37 1857.93 1378.37 1863.15 L1378.37 1883.3 L1372.48 1883.3 L1372.48 1861.98 Q1372.48 1856.82 1370.67 1854.37 Q1368.85 1851.89 1365.06 1851.89 Q1360.58 1851.89 1357.94 1854.94 Q1355.29 1857.97 1355.29 1863.15 L1355.29 1883.3 L1349.4 1883.3 L1349.4 1847.65 L1355.29 1847.65 L1355.29 1853.19 Q1357.3 1849.91 1360.1 1848.35 Q1362.9 1846.79 1366.75 1846.79 Q1370.63 1846.79 1373.34 1848.77 Q1376.08 1850.74 1377.38 1854.5 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M1443.62 1864.01 L1443.62 1866.88 L1416.69 1866.88 Q1417.07 1872.92 1420.32 1876.11 Q1423.6 1879.26 1429.42 1879.26 Q1432.8 1879.26 1435.95 1878.43 Q1439.13 1877.6 1442.25 1875.95 L1442.25 1881.49 Q1439.1 1882.82 1435.79 1883.52 Q1432.48 1884.22 1429.07 1884.22 Q1420.54 1884.22 1415.54 1879.26 Q1410.58 1874.29 1410.58 1865.83 Q1410.58 1857.07 1415.29 1851.95 Q1420.03 1846.79 1428.05 1846.79 Q1435.25 1846.79 1439.42 1851.44 Q1443.62 1856.06 1443.62 1864.01 M1437.76 1862.29 Q1437.7 1857.49 1435.06 1854.62 Q1432.45 1851.76 1428.12 1851.76 Q1423.22 1851.76 1420.26 1854.53 Q1417.33 1857.3 1416.88 1862.33 L1437.76 1862.29 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M1474.2 1835.78 L1480.63 1835.78 L1480.63 1877.89 L1503.77 1877.89 L1503.77 1883.3 L1474.2 1883.3 L1474.2 1835.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M1526.47 1865.38 Q1519.37 1865.38 1516.63 1867 Q1513.89 1868.63 1513.89 1872.54 Q1513.89 1875.66 1515.93 1877.51 Q1518 1879.32 1521.53 1879.32 Q1526.4 1879.32 1529.33 1875.88 Q1532.29 1872.42 1532.29 1866.69 L1532.29 1865.38 L1526.47 1865.38 M1538.15 1862.96 L1538.15 1883.3 L1532.29 1883.3 L1532.29 1877.89 Q1530.29 1881.14 1527.29 1882.7 Q1524.3 1884.22 1519.97 1884.22 Q1514.5 1884.22 1511.25 1881.17 Q1508.04 1878.08 1508.04 1872.92 Q1508.04 1866.91 1512.05 1863.85 Q1516.09 1860.8 1524.08 1860.8 L1532.29 1860.8 L1532.29 1860.23 Q1532.29 1856.18 1529.62 1853.99 Q1526.98 1851.76 1522.17 1851.76 Q1519.11 1851.76 1516.22 1852.49 Q1513.32 1853.22 1510.65 1854.69 L1510.65 1849.28 Q1513.86 1848.03 1516.89 1847.43 Q1519.91 1846.79 1522.78 1846.79 Q1530.51 1846.79 1534.33 1850.8 Q1538.15 1854.81 1538.15 1862.96 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M1573.67 1865.06 Q1573.67 1858.7 1571.03 1855.2 Q1568.42 1851.7 1563.67 1851.7 Q1558.96 1851.7 1556.32 1855.2 Q1553.71 1858.7 1553.71 1865.06 Q1553.71 1871.4 1556.32 1874.9 Q1558.96 1878.4 1563.67 1878.4 Q1568.42 1878.4 1571.03 1874.9 Q1573.67 1871.4 1573.67 1865.06 M1579.53 1878.88 Q1579.53 1887.98 1575.48 1892.4 Q1571.44 1896.86 1563.1 1896.86 Q1560.01 1896.86 1557.28 1896.38 Q1554.54 1895.94 1551.96 1894.98 L1551.96 1889.28 Q1554.54 1890.69 1557.05 1891.35 Q1559.57 1892.02 1562.18 1892.02 Q1567.94 1892.02 1570.8 1889 Q1573.67 1886.01 1573.67 1879.93 L1573.67 1877.03 Q1571.85 1880.18 1569.02 1881.74 Q1566.19 1883.3 1562.24 1883.3 Q1555.69 1883.3 1551.68 1878.3 Q1547.66 1873.31 1547.66 1865.06 Q1547.66 1856.79 1551.68 1851.79 Q1555.69 1846.79 1562.24 1846.79 Q1566.19 1846.79 1569.02 1848.35 Q1571.85 1849.91 1573.67 1853.06 L1573.67 1847.65 L1579.53 1847.65 L1579.53 1878.88 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip390)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.141,1734.12 226.141,47.2441 "/>
<polyline clip-path="url(#clip390)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.141,1497.09 245.038,1497.09 "/>
<polyline clip-path="url(#clip390)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.141,1230.66 245.038,1230.66 "/>
<polyline clip-path="url(#clip390)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.141,964.222 245.038,964.222 "/>
<polyline clip-path="url(#clip390)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.141,697.785 245.038,697.785 "/>
<polyline clip-path="url(#clip390)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.141,431.349 245.038,431.349 "/>
<polyline clip-path="url(#clip390)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.141,164.913 245.038,164.913 "/>
<path clip-path="url(#clip390)" d="M128.645 1482.89 Q125.034 1482.89 123.205 1486.46 Q121.4 1490 121.4 1497.13 Q121.4 1504.24 123.205 1507.8 Q125.034 1511.34 128.645 1511.34 Q132.279 1511.34 134.085 1507.8 Q135.913 1504.24 135.913 1497.13 Q135.913 1490 134.085 1486.46 Q132.279 1482.89 128.645 1482.89 M128.645 1479.19 Q134.455 1479.19 137.511 1483.8 Q140.589 1488.38 140.589 1497.13 Q140.589 1505.86 137.511 1510.46 Q134.455 1515.05 128.645 1515.05 Q122.835 1515.05 119.756 1510.46 Q116.701 1505.86 116.701 1497.13 Q116.701 1488.38 119.756 1483.8 Q122.835 1479.19 128.645 1479.19 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M148.807 1508.49 L153.691 1508.49 L153.691 1514.37 L148.807 1514.37 L148.807 1508.49 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M173.876 1482.89 Q170.265 1482.89 168.436 1486.46 Q166.631 1490 166.631 1497.13 Q166.631 1504.24 168.436 1507.8 Q170.265 1511.34 173.876 1511.34 Q177.51 1511.34 179.316 1507.8 Q181.145 1504.24 181.145 1497.13 Q181.145 1490 179.316 1486.46 Q177.51 1482.89 173.876 1482.89 M173.876 1479.19 Q179.686 1479.19 182.742 1483.8 Q185.821 1488.38 185.821 1497.13 Q185.821 1505.86 182.742 1510.46 Q179.686 1515.05 173.876 1515.05 Q168.066 1515.05 164.987 1510.46 Q161.932 1505.86 161.932 1497.13 Q161.932 1488.38 164.987 1483.8 Q168.066 1479.19 173.876 1479.19 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M129.872 1216.46 Q126.261 1216.46 124.432 1220.02 Q122.626 1223.56 122.626 1230.69 Q122.626 1237.8 124.432 1241.36 Q126.261 1244.91 129.872 1244.91 Q133.506 1244.91 135.312 1241.36 Q137.14 1237.8 137.14 1230.69 Q137.14 1223.56 135.312 1220.02 Q133.506 1216.46 129.872 1216.46 M129.872 1212.75 Q135.682 1212.75 138.737 1217.36 Q141.816 1221.94 141.816 1230.69 Q141.816 1239.42 138.737 1244.03 Q135.682 1248.61 129.872 1248.61 Q124.062 1248.61 120.983 1244.03 Q117.927 1239.42 117.927 1230.69 Q117.927 1221.94 120.983 1217.36 Q124.062 1212.75 129.872 1212.75 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M150.034 1242.06 L154.918 1242.06 L154.918 1247.94 L150.034 1247.94 L150.034 1242.06 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M165.913 1244 L173.552 1244 L173.552 1217.64 L165.242 1219.3 L165.242 1215.04 L173.506 1213.38 L178.182 1213.38 L178.182 1244 L185.821 1244 L185.821 1247.94 L165.913 1247.94 L165.913 1244 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M130.242 950.02 Q126.631 950.02 124.802 953.585 Q122.997 957.127 122.997 964.256 Q122.997 971.363 124.802 974.928 Q126.631 978.469 130.242 978.469 Q133.876 978.469 135.682 974.928 Q137.511 971.363 137.511 964.256 Q137.511 957.127 135.682 953.585 Q133.876 950.02 130.242 950.02 M130.242 946.317 Q136.052 946.317 139.108 950.923 Q142.187 955.506 142.187 964.256 Q142.187 972.983 139.108 977.59 Q136.052 982.173 130.242 982.173 Q124.432 982.173 121.353 977.59 Q118.298 972.983 118.298 964.256 Q118.298 955.506 121.353 950.923 Q124.432 946.317 130.242 946.317 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M150.404 975.622 L155.288 975.622 L155.288 981.502 L150.404 981.502 L150.404 975.622 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M169.501 977.566 L185.821 977.566 L185.821 981.502 L163.876 981.502 L163.876 977.566 Q166.538 974.812 171.122 970.182 Q175.728 965.529 176.909 964.187 Q179.154 961.664 180.034 959.928 Q180.936 958.168 180.936 956.479 Q180.936 953.724 178.992 951.988 Q177.071 950.252 173.969 950.252 Q171.77 950.252 169.316 951.016 Q166.885 951.78 164.108 953.33 L164.108 948.608 Q166.932 947.474 169.385 946.895 Q171.839 946.317 173.876 946.317 Q179.247 946.317 182.441 949.002 Q185.635 951.687 185.635 956.178 Q185.635 958.307 184.825 960.229 Q184.038 962.127 181.932 964.719 Q181.353 965.391 178.251 968.608 Q175.149 971.803 169.501 977.566 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M129.293 683.584 Q125.682 683.584 123.853 687.149 Q122.048 690.691 122.048 697.82 Q122.048 704.927 123.853 708.491 Q125.682 712.033 129.293 712.033 Q132.927 712.033 134.733 708.491 Q136.562 704.927 136.562 697.82 Q136.562 690.691 134.733 687.149 Q132.927 683.584 129.293 683.584 M129.293 679.88 Q135.103 679.88 138.159 684.487 Q141.237 689.07 141.237 697.82 Q141.237 706.547 138.159 711.153 Q135.103 715.737 129.293 715.737 Q123.483 715.737 120.404 711.153 Q117.349 706.547 117.349 697.82 Q117.349 689.07 120.404 684.487 Q123.483 679.88 129.293 679.88 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M149.455 709.186 L154.339 709.186 L154.339 715.065 L149.455 715.065 L149.455 709.186 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M178.691 696.431 Q182.047 697.149 183.922 699.417 Q185.821 701.686 185.821 705.019 Q185.821 710.135 182.302 712.936 Q178.784 715.737 172.302 715.737 Q170.126 715.737 167.811 715.297 Q165.52 714.88 163.066 714.024 L163.066 709.51 Q165.01 710.644 167.325 711.223 Q169.64 711.802 172.163 711.802 Q176.561 711.802 178.853 710.065 Q181.168 708.329 181.168 705.019 Q181.168 701.964 179.015 700.251 Q176.885 698.515 173.066 698.515 L169.038 698.515 L169.038 694.672 L173.251 694.672 Q176.7 694.672 178.529 693.306 Q180.358 691.917 180.358 689.325 Q180.358 686.663 178.459 685.251 Q176.584 683.816 173.066 683.816 Q171.145 683.816 168.946 684.232 Q166.747 684.649 164.108 685.529 L164.108 681.362 Q166.77 680.621 169.085 680.251 Q171.422 679.88 173.483 679.88 Q178.807 679.88 181.909 682.311 Q185.01 684.718 185.01 688.839 Q185.01 691.709 183.367 693.7 Q181.723 695.667 178.691 696.431 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M128.159 417.148 Q124.548 417.148 122.719 420.713 Q120.913 424.254 120.913 431.384 Q120.913 438.49 122.719 442.055 Q124.548 445.597 128.159 445.597 Q131.793 445.597 133.599 442.055 Q135.427 438.49 135.427 431.384 Q135.427 424.254 133.599 420.713 Q131.793 417.148 128.159 417.148 M128.159 413.444 Q133.969 413.444 137.025 418.051 Q140.103 422.634 140.103 431.384 Q140.103 440.111 137.025 444.717 Q133.969 449.301 128.159 449.301 Q122.349 449.301 119.27 444.717 Q116.214 440.111 116.214 431.384 Q116.214 422.634 119.27 418.051 Q122.349 413.444 128.159 413.444 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M148.321 442.75 L153.205 442.75 L153.205 448.629 L148.321 448.629 L148.321 442.75 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M176.237 418.143 L164.432 436.592 L176.237 436.592 L176.237 418.143 M175.01 414.069 L180.89 414.069 L180.89 436.592 L185.821 436.592 L185.821 440.481 L180.89 440.481 L180.89 448.629 L176.237 448.629 L176.237 440.481 L160.636 440.481 L160.636 435.967 L175.01 414.069 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M129.64 150.712 Q126.029 150.712 124.201 154.277 Q122.395 157.818 122.395 164.948 Q122.395 172.054 124.201 175.619 Q126.029 179.161 129.64 179.161 Q133.275 179.161 135.08 175.619 Q136.909 172.054 136.909 164.948 Q136.909 157.818 135.08 154.277 Q133.275 150.712 129.64 150.712 M129.64 147.008 Q135.45 147.008 138.506 151.615 Q141.585 156.198 141.585 164.948 Q141.585 173.675 138.506 178.281 Q135.45 182.864 129.64 182.864 Q123.83 182.864 120.751 178.281 Q117.696 173.675 117.696 164.948 Q117.696 156.198 120.751 151.615 Q123.83 147.008 129.64 147.008 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M149.802 176.314 L154.686 176.314 L154.686 182.193 L149.802 182.193 L149.802 176.314 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M164.918 147.633 L183.274 147.633 L183.274 151.568 L169.2 151.568 L169.2 160.041 Q170.219 159.693 171.237 159.531 Q172.256 159.346 173.274 159.346 Q179.061 159.346 182.441 162.517 Q185.821 165.689 185.821 171.105 Q185.821 176.684 182.348 179.786 Q178.876 182.864 172.557 182.864 Q170.381 182.864 168.112 182.494 Q165.867 182.124 163.46 181.383 L163.46 176.684 Q165.543 177.818 167.765 178.374 Q169.987 178.929 172.464 178.929 Q176.469 178.929 178.807 176.823 Q181.145 174.716 181.145 171.105 Q181.145 167.494 178.807 165.388 Q176.469 163.281 172.464 163.281 Q170.589 163.281 168.714 163.698 Q166.862 164.115 164.918 164.994 L164.918 147.633 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M19.402 1241.62 L37.2578 1241.62 L37.2578 1233.54 Q37.2578 1229.05 34.9343 1226.6 Q32.6108 1224.15 28.314 1224.15 Q24.0489 1224.15 21.7255 1226.6 Q19.402 1229.05 19.402 1233.54 L19.402 1241.62 M14.1184 1248.05 L14.1184 1233.54 Q14.1184 1225.55 17.7469 1221.47 Q21.3435 1217.37 28.314 1217.37 Q35.3481 1217.37 38.9447 1221.47 Q42.5413 1225.55 42.5413 1233.54 L42.5413 1241.62 L61.6384 1241.62 L61.6384 1248.05 L14.1184 1248.05 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M43.719 1195.73 Q43.719 1202.82 45.3422 1205.56 Q46.9655 1208.3 50.8804 1208.3 Q53.9996 1208.3 55.8457 1206.26 Q57.6599 1204.19 57.6599 1200.66 Q57.6599 1195.79 54.2224 1192.86 Q50.7531 1189.9 45.0239 1189.9 L43.719 1189.9 L43.719 1195.73 M41.3 1184.04 L61.6384 1184.04 L61.6384 1189.9 L56.2276 1189.9 Q59.4741 1191.91 61.0337 1194.9 Q62.5615 1197.89 62.5615 1202.22 Q62.5615 1207.69 59.5059 1210.94 Q56.4186 1214.15 51.2623 1214.15 Q45.2467 1214.15 42.1912 1210.14 Q39.1357 1206.1 39.1357 1198.11 L39.1357 1189.9 L38.5628 1189.9 Q34.5205 1189.9 32.3244 1192.57 Q30.0964 1195.22 30.0964 1200.02 Q30.0964 1203.08 30.8284 1205.97 Q31.5605 1208.87 33.0246 1211.54 L27.6137 1211.54 Q26.3724 1208.33 25.7677 1205.31 Q25.1311 1202.28 25.1311 1199.42 Q25.1311 1191.68 29.1415 1187.86 Q33.1519 1184.04 41.3 1184.04 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M31.465 1151.32 Q30.8921 1152.31 30.6375 1153.49 Q30.351 1154.63 30.351 1156.04 Q30.351 1161 33.5975 1163.67 Q36.8122 1166.32 42.8596 1166.32 L61.6384 1166.32 L61.6384 1172.2 L25.9905 1172.2 L25.9905 1166.32 L31.5287 1166.32 Q28.2821 1164.47 26.7225 1161.51 Q25.1311 1158.55 25.1311 1154.32 Q25.1311 1153.71 25.2266 1152.98 Q25.2903 1152.25 25.4494 1151.36 L31.465 1151.32 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M15.869 1139.39 L25.9905 1139.39 L25.9905 1127.33 L30.542 1127.33 L30.542 1139.39 L49.8937 1139.39 Q54.2542 1139.39 55.4955 1138.21 Q56.7368 1137 56.7368 1133.34 L56.7368 1127.33 L61.6384 1127.33 L61.6384 1133.34 Q61.6384 1140.12 59.124 1142.7 Q56.5777 1145.28 49.8937 1145.28 L30.542 1145.28 L30.542 1149.57 L25.9905 1149.57 L25.9905 1145.28 L15.869 1145.28 L15.869 1139.39 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M25.9905 1119.62 L25.9905 1113.77 L61.6384 1113.77 L61.6384 1119.62 L25.9905 1119.62 M12.1132 1119.62 L12.1132 1113.77 L19.5293 1113.77 L19.5293 1119.62 L12.1132 1119.62 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M43.719 1085.31 Q43.719 1092.41 45.3422 1095.15 Q46.9655 1097.88 50.8804 1097.88 Q53.9996 1097.88 55.8457 1095.85 Q57.6599 1093.78 57.6599 1090.25 Q57.6599 1085.38 54.2224 1082.45 Q50.7531 1079.49 45.0239 1079.49 L43.719 1079.49 L43.719 1085.31 M41.3 1073.63 L61.6384 1073.63 L61.6384 1079.49 L56.2276 1079.49 Q59.4741 1081.49 61.0337 1084.48 Q62.5615 1087.48 62.5615 1091.81 Q62.5615 1097.28 59.5059 1100.53 Q56.4186 1103.74 51.2623 1103.74 Q45.2467 1103.74 42.1912 1099.73 Q39.1357 1095.69 39.1357 1087.7 L39.1357 1079.49 L38.5628 1079.49 Q34.5205 1079.49 32.3244 1082.16 Q30.0964 1084.8 30.0964 1089.61 Q30.0964 1092.66 30.8284 1095.56 Q31.5605 1098.46 33.0246 1101.13 L27.6137 1101.13 Q26.3724 1097.92 25.7677 1094.89 Q25.1311 1091.87 25.1311 1089 Q25.1311 1081.27 29.1415 1077.45 Q33.1519 1073.63 41.3 1073.63 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M12.1132 1061.57 L12.1132 1055.71 L61.6384 1055.71 L61.6384 1061.57 L12.1132 1061.57 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M20.4523 1006.6 L44.1009 1015.32 L44.1009 997.847 L20.4523 1006.6 M14.1184 1010.23 L14.1184 1002.94 L61.6384 984.829 L61.6384 991.513 L49.4481 995.842 L49.4481 1017.26 L61.6384 1021.59 L61.6384 1028.37 L14.1184 1010.23 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M47.5702 978.75 L25.9905 978.75 L25.9905 972.894 L47.3474 972.894 Q52.4082 972.894 54.9545 970.92 Q57.4689 968.947 57.4689 965 Q57.4689 960.258 54.4452 957.52 Q51.4215 954.751 46.2016 954.751 L25.9905 954.751 L25.9905 948.895 L61.6384 948.895 L61.6384 954.751 L56.1639 954.751 Q59.4104 956.884 61.0019 959.717 Q62.5615 962.518 62.5615 966.241 Q62.5615 972.384 58.742 975.567 Q54.9226 978.75 47.5702 978.75 M25.1311 964.013 L25.1311 964.013 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M15.869 931.039 L25.9905 931.039 L25.9905 918.976 L30.542 918.976 L30.542 931.039 L49.8937 931.039 Q54.2542 931.039 55.4955 929.861 Q56.7368 928.652 56.7368 924.992 L56.7368 918.976 L61.6384 918.976 L61.6384 924.992 Q61.6384 931.771 59.124 934.349 Q56.5777 936.927 49.8937 936.927 L30.542 936.927 L30.542 941.224 L25.9905 941.224 L25.9905 936.927 L15.869 936.927 L15.869 931.039 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M30.0964 897.46 Q30.0964 902.171 33.7885 904.908 Q37.4488 907.645 43.8463 907.645 Q50.2438 907.645 53.9359 904.94 Q57.5962 902.202 57.5962 897.46 Q57.5962 892.781 53.9041 890.044 Q50.212 887.307 43.8463 887.307 Q37.5124 887.307 33.8203 890.044 Q30.0964 892.781 30.0964 897.46 M25.1311 897.46 Q25.1311 889.821 30.0964 885.461 Q35.0616 881.1 43.8463 881.1 Q52.5991 881.1 57.5962 885.461 Q62.5615 889.821 62.5615 897.46 Q62.5615 905.131 57.5962 909.491 Q52.5991 913.82 43.8463 913.82 Q35.0616 913.82 30.0964 909.491 Q25.1311 905.131 25.1311 897.46 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M27.3591 845.739 L32.8336 845.739 Q31.465 848.221 30.7966 850.736 Q30.0964 853.218 30.0964 855.765 Q30.0964 861.462 33.7248 864.613 Q37.3214 867.764 43.8463 867.764 Q50.3711 867.764 53.9996 864.613 Q57.5962 861.462 57.5962 855.765 Q57.5962 853.218 56.9278 850.736 Q56.2276 848.221 54.859 845.739 L60.2698 845.739 Q61.4156 848.189 61.9886 850.831 Q62.5615 853.441 62.5615 856.401 Q62.5615 864.454 57.5007 869.196 Q52.44 873.939 43.8463 873.939 Q35.1253 873.939 30.1282 869.164 Q25.1311 864.358 25.1311 856.019 Q25.1311 853.314 25.704 850.736 Q26.2451 848.158 27.3591 845.739 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M30.0964 821.74 Q30.0964 826.451 33.7885 829.188 Q37.4488 831.925 43.8463 831.925 Q50.2438 831.925 53.9359 829.22 Q57.5962 826.482 57.5962 821.74 Q57.5962 817.061 53.9041 814.324 Q50.212 811.587 43.8463 811.587 Q37.5124 811.587 33.8203 814.324 Q30.0964 817.061 30.0964 821.74 M25.1311 821.74 Q25.1311 814.101 30.0964 809.741 Q35.0616 805.38 43.8463 805.38 Q52.5991 805.38 57.5962 809.741 Q62.5615 814.101 62.5615 821.74 Q62.5615 829.411 57.5962 833.771 Q52.5991 838.1 43.8463 838.1 Q35.0616 838.1 30.0964 833.771 Q25.1311 829.411 25.1311 821.74 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M31.465 775.016 Q30.8921 776.002 30.6375 777.18 Q30.351 778.326 30.351 779.726 Q30.351 784.692 33.5975 787.365 Q36.8122 790.007 42.8596 790.007 L61.6384 790.007 L61.6384 795.895 L25.9905 795.895 L25.9905 790.007 L31.5287 790.007 Q28.2821 788.161 26.7225 785.201 Q25.1311 782.241 25.1311 778.008 Q25.1311 777.403 25.2266 776.671 Q25.2903 775.939 25.4494 775.047 L31.465 775.016 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M31.465 749.362 Q30.8921 750.349 30.6375 751.526 Q30.351 752.672 30.351 754.072 Q30.351 759.038 33.5975 761.711 Q36.8122 764.353 42.8596 764.353 L61.6384 764.353 L61.6384 770.241 L25.9905 770.241 L25.9905 764.353 L31.5287 764.353 Q28.2821 762.507 26.7225 759.547 Q25.1311 756.587 25.1311 752.354 Q25.1311 751.749 25.2266 751.017 Q25.2903 750.285 25.4494 749.394 L31.465 749.362 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M42.3504 714.159 L45.2149 714.159 L45.2149 741.086 Q51.2623 740.704 54.4452 737.458 Q57.5962 734.18 57.5962 728.355 Q57.5962 724.981 56.7687 721.83 Q55.9411 718.647 54.2861 715.528 L59.8242 715.528 Q61.161 718.679 61.8612 721.989 Q62.5615 725.299 62.5615 728.705 Q62.5615 737.235 57.5962 742.232 Q52.631 747.198 44.1646 747.198 Q35.4117 747.198 30.2873 742.487 Q25.1311 737.744 25.1311 729.724 Q25.1311 722.53 29.7781 718.361 Q34.3932 714.159 42.3504 714.159 M40.6316 720.016 Q35.8255 720.08 32.9609 722.721 Q30.0964 725.331 30.0964 729.66 Q30.0964 734.562 32.8654 737.522 Q35.6345 740.45 40.6634 740.895 L40.6316 720.016 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M12.1132 704.547 L12.1132 698.691 L61.6384 698.691 L61.6384 704.547 L12.1132 704.547 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M43.719 670.236 Q43.719 677.334 45.3422 680.071 Q46.9655 682.808 50.8804 682.808 Q53.9996 682.808 55.8457 680.771 Q57.6599 678.703 57.6599 675.17 Q57.6599 670.3 54.2224 667.372 Q50.7531 664.411 45.0239 664.411 L43.719 664.411 L43.719 670.236 M41.3 658.555 L61.6384 658.555 L61.6384 664.411 L56.2276 664.411 Q59.4741 666.417 61.0337 669.409 Q62.5615 672.4 62.5615 676.729 Q62.5615 682.204 59.5059 685.45 Q56.4186 688.665 51.2623 688.665 Q45.2467 688.665 42.1912 684.654 Q39.1357 680.612 39.1357 672.623 L39.1357 664.411 L38.5628 664.411 Q34.5205 664.411 32.3244 667.085 Q30.0964 669.727 30.0964 674.533 Q30.0964 677.589 30.8284 680.485 Q31.5605 683.381 33.0246 686.055 L27.6137 686.055 Q26.3724 682.84 25.7677 679.817 Q25.1311 676.793 25.1311 673.928 Q25.1311 666.194 29.1415 662.374 Q33.1519 658.555 41.3 658.555 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M15.869 640.699 L25.9905 640.699 L25.9905 628.636 L30.542 628.636 L30.542 640.699 L49.8937 640.699 Q54.2542 640.699 55.4955 639.522 Q56.7368 638.312 56.7368 634.652 L56.7368 628.636 L61.6384 628.636 L61.6384 634.652 Q61.6384 641.431 59.124 644.009 Q56.5777 646.588 49.8937 646.588 L30.542 646.588 L30.542 650.884 L25.9905 650.884 L25.9905 646.588 L15.869 646.588 L15.869 640.699 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M25.9905 620.934 L25.9905 615.077 L61.6384 615.077 L61.6384 620.934 L25.9905 620.934 M12.1132 620.934 L12.1132 615.077 L19.5293 615.077 L19.5293 620.934 L12.1132 620.934 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M30.0964 589.01 Q30.0964 593.72 33.7885 596.458 Q37.4488 599.195 43.8463 599.195 Q50.2438 599.195 53.9359 596.489 Q57.5962 593.752 57.5962 589.01 Q57.5962 584.331 53.9041 581.594 Q50.212 578.856 43.8463 578.856 Q37.5124 578.856 33.8203 581.594 Q30.0964 584.331 30.0964 589.01 M25.1311 589.01 Q25.1311 581.371 30.0964 577.01 Q35.0616 572.65 43.8463 572.65 Q52.5991 572.65 57.5962 577.01 Q62.5615 581.371 62.5615 589.01 Q62.5615 596.68 57.5962 601.041 Q52.5991 605.37 43.8463 605.37 Q35.0616 605.37 30.0964 601.041 Q25.1311 596.68 25.1311 589.01 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip390)" d="M40.1224 533.31 L61.6384 533.31 L61.6384 539.166 L40.3133 539.166 Q35.2526 539.166 32.7381 541.14 Q30.2237 543.113 30.2237 547.06 Q30.2237 551.802 33.2474 554.539 Q36.2711 557.277 41.491 557.277 L61.6384 557.277 L61.6384 563.165 L25.9905 563.165 L25.9905 557.277 L31.5287 557.277 Q28.314 555.176 26.7225 552.343 Q25.1311 549.479 25.1311 545.755 Q25.1311 539.612 28.9505 536.461 Q32.7381 533.31 40.1224 533.31 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><circle clip-path="url(#clip392)" cx="294.479" cy="94.9858" r="18" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip392)" cx="863.963" cy="1250.24" r="18" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip392)" cx="1433.45" cy="1360.01" r="18" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip392)" cx="2002.93" cy="1348.59" r="18" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<circle clip-path="url(#clip392)" cx="2572.42" cy="1686.38" r="18" fill="#009af9" fill-rule="evenodd" fill-opacity="1" stroke="#000000" stroke-opacity="1" stroke-width="3.84"/>
<polyline clip-path="url(#clip392)" style="stroke:#009af9; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="294.479,1497.09 294.479,94.9858 "/>
<polyline clip-path="url(#clip392)" style="stroke:#009af9; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="863.963,1497.09 863.963,1250.24 "/>
<polyline clip-path="url(#clip392)" style="stroke:#009af9; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1433.45,1497.09 1433.45,1360.01 "/>
<polyline clip-path="url(#clip392)" style="stroke:#009af9; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2002.93,1497.09 2002.93,1348.59 "/>
<polyline clip-path="url(#clip392)" style="stroke:#009af9; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2572.42,1497.09 2572.42,1686.38 "/>
</svg>
<p>(b) Partial Autocorrelation Plot</p>
</div>
</div></td>
</tr>
</tbody>
</table>

Figure 2: Diagnostics for the model residuals.

We can see that while the residuals appear pretty-well described by a
Gaussian distribution based on the Q-Q plot, they have a reasonable
lag-1 autocorrelation of about 0.53, so this may be an inappropriate
assumption. We should try an AR(1) discrepancy model to account for
this.

### Problem 2

First, load the data, find the weather-related residuals, and plot. To
load and process the data, we will use the `DataFramesMeta.jl` package,
which lets us string together commands in a convenient way. This is
completely optional, however.

In [1]:
tide_dat = CSV.read(joinpath("data", "norfolk-hourly-surge-2015.csv"), DataFrame)
surge_resids = tide_dat[:, 5] - tide_dat[:, 3]
plot(surge_resids; ylabel="Gauge Measurement (m)", label="Weather-Related Residual", legend=:topleft, xlabel="Hour Number")

Let’s look at the autocorrelation function of the residuals to determine
what the autoregressive lag should be.

In [1]:
resid_acf = pacf(surge_resids, 1:5)
plot(resid_acf , marker=:circle, line=:stem, linewidth=3, markersize=5, legend=false, ylabel="Partial Autocorrelation", xlabel="Time Lag")

We can see that there is a *very* strong autocorrelation at lag 1, but
not much additional autocorrelation introduced at lag 2, so we will use
an AR(1) model (though you might be justified in using an AR(2) model,
as the lag-2 partial autocorrelation is not completely negligible).

This AR(1) model is
$$y_t = \rho y_{t+1} + \varepsilon, \quad \varepsilon \sim N(0, \sigma).$$
Let’s implement this model and fit it. We’ll specify the likelihood
based on whitening the residuals and computing the individual normal
likelihoods of the resulting terms, but you should get the same response
if you used the joint likelihood of the entire time series.

In [1]:
function llik_ar(params, y)
    ρ, σ = params
    ll = 0 # initialize log-likelihood counter
    ll += logpdf(Normal(0, σ/sqrt(1-ρ^2)), y[1])
    for i = 1:length(y)-1
        residuals_whitened = y[i+1] - ρ * y[i]
        ll += logpdf(Normal(0, σ), residuals_whitened)
    end
    return ll
end

# set up lower and upper bounds for the parameters for the optimization
lbds = [0.0, 0.0]
ubds = [1.0, 5.0]
p0 = [0.5, 2.0]
p_tide_mle = Optim.optimize(p -> -llik_ar(p, surge_resids), lbds, ubds, p0).minimizer

2-element Vector{Float64}:
 0.9956392098239767
 0.019663589610916302

Now we can simulate these residuals and add them back to the NOAA model
predictions using the `mapslices()` function. The resulting histogram of
the maximum values is shown in
<a href="#fig-p2-histogram" class="quarto-xref">Figure 5</a>.

In [1]:
# T is the length of the simulated series, N the number of simulations
function simulate_ar1(ρ, σ, T, N)
    y = zeros(T, N) # initialize storage
    y[1, :] = rand(Normal(0, σ/sqrt(1-ρ^2)), N) # sample initial value
    # simulate the remaining residuals
    for t = 2:T
        y[t, :] = ρ * y[t-1, :] + rand(Normal(0, σ), N)
    end
    return y
end

# simulate new observations
simulated_resids = simulate_ar1(p_tide_mle[1], p_tide_mle[2], length(surge_resids), 10_000)
simulated_obs = mapslices(col -> col + tide_dat[:, 3], simulated_resids, dims=1)
# plot histogram
max_obs = [maximum(simulated_obs[:, n]) for n in 1:size(simulated_obs, 2)]
histogram(max_obs, xlabel="Maximum Simulated Gauge Level (m)", ylabel="Count", label="Simulations")
vline!([maximum(tide_dat[:, 5])], color=:red, linestyle=:dash, label="Observation")

This distribution fails to capture the observed maximum 1.98, which
could be due to a model structural problem (we might want to allow for
more extreme AR(1) innovations than the normal distribution of
$\varepsilon_t$) or just the unlikeliness of that one observation.

## Problem 3

Loading the data (note the delimiter is now a semi-colon):

In [1]:
dat = CSV.read(joinpath("data", "salamanders.csv"), DataFrame, delim=";")

The first model, using `PCTCOVER` as a predictor, can be specified as
the following (using the standard log link function for Poisson
regression):

$$
\begin{aligned}
S &\sim \text{Poisson}(\lambda) \\
\log(\lambda) &= a PC + b
\end{aligned}
$$

As noted in the problem, we will need to standardize the `PCTCOVER`
variable, as its range is much larger than the salamander counts.

In [1]:
function salamander_pcover(p, counts, pctcover) 
    a, b = p
    λ = exp.(a * pctcover .+ b)
    ll = sum(logpdf.(Poisson.(λ), counts))
    return ll
end

lb = [-10.0, -50.0]
ub = [10.0, 50.0]
p0 = [0.0, 0.0]

# function to make this more convenient
stdz(x) = (x .- mean(x)) / std(x)

result = optimize(p -> -salamander_pcover(p, dat.SALAMAN, stdz(dat.PCTCOVER)), lb, ub, p0)
pcover_mle = result.minimizer
@show pcover_mle;

pcover_mle = [1.1595061318756, 0.42951129115546005]

To translate these estimates to expected values and uncertainty
intervals, we now simulate observations of salamanders based on this
model.

In [1]:
function sim_salamanders_pcover(params, pctcover, n)
    a, b = params
    λ = exp.(a * pctcover .+ b)
    sal_pred = zeros(n, length(pctcover))
    for i = 1:length(pctcover)
        sal_pred[:, i] = rand(Poisson(λ[i]), n)
    end
    return sal_pred
end

sim_pcover = sim_salamanders_pcover(pcover_mle, -2:0.01:2, 10_000)

pcover_q = mapslices(col -> quantile(col, [0.05, 0.5, 0.95]), sim_pcover; dims=1) 
plot(-2:0.01:2, pcover_q[2, :], linewidth=3, ribbon=(pcover_q[2, :] - pcover_q[1, :], pcover_q[3, :] - pcover_q[2, :]), fillalpha=0.5, xlabel="Standardized Percent Ground Cover", ylabel="Salamander Count", label="Simulations")
scatter!(stdz(dat.PCTCOVER), dat.SALAMAN, label="Observations")

The model seems to do ok for lower levels of percent cover, but does not
account for the level of dispersion at higher levels; this might suggest
that a Poisson model is not ideal, but we could try a negative binomial
regression instead.

Let’s see if we can improve the model using the `FORESTAGE` predictor.
We’ll try to add this in as a linear predictor.

In [1]:
function salamander_fage(p, counts, pctcover, forestage) 
    a1, a2, b = p
    λ = exp.(a1 * pctcover + a2 * forestage .+ b)
    ll = sum(logpdf.(Poisson.(λ), counts))
    return ll
end

lb = [-10.0, -10.0, -50.0]
ub = [10.0, 10.0, 50.0]
p0 = [0.0, 0.0, 0.0]

result = optimize(p -> -salamander_fage(p, dat.SALAMAN, stdz(dat.PCTCOVER), stdz(dat.FORESTAGE)), lb, ub, p0)
fage_mle = result.minimizer
@show fage_mle;

fage_mle = [1.16246761350599, -0.004084430295909137, 0.4293187208222515]

We can see that the `FORESTAGE` predictor variable is effectively zero,
which means it does not add much to the prediction. Why is this? The
correlation between `PCTCOVER` and `FORESTAGE` is quite high: 0.63,
which means most of the information in `FORESTAGE` has already been
included in the `PCTCOVER` model. It makes sense that `PCTCOVER` is
valuable: salamanders are likely to seek areas with high levels of
ground cover, while the age of the forest is only meaningful to the
extent that older forests may have more ground cover (hence the high
correlation).

### Problem 4

Loading the data (once again, the file is delimited by semi-colons):

In [1]:
dat = CSV.read(joinpath("data", "Hurricanes.csv"), DataFrame, delim=";")

Our interpretation of the hypothesis results in the following model
specification:

$$
\begin{aligned}
D &\sim \text{Poisson}(\lambda) \\
\log(\lambda) &= a_1 FN + a_2 MP + b
\end{aligned}
$$

We need to standardized the pressure predictor; we may as well
standardize the femininity predictor as well, though it turns out this
is less necessary (you’ll just get slightly different regression
coefficients if you don’t).

In [1]:
function hurricane_pressure(p, counts, fem, pressure) 
    a1, a2, b = p
    λ = exp.(a1 * fem + a2 * pressure .+ b)
    ll = sum(logpdf.(Poisson.(λ), counts))
    return ll
end

lb = [-10.0, -10.0, -50.0]
ub = [10.0, 10.0, 50.0]
p0 = [5.0, 5.0, 0.0]

result = optimize(p -> -hurricane_pressure(p, dat.deaths, stdz(dat.femininity), stdz(dat.min_pressure)), lb, ub, p0)
p_pressure = result.minimizer
@show p_pressure;

p_pressure = [0.20728325721324137, -0.7152762599042397, 2.7368598941753794]

In [1]:
function sim_hurricanes(p, femininity, damage,  n)
    a1, a2, b = p
    λ = exp.(a1 * femininity + a2 * damage .+ b)
    hur_pred = zeros(n, length(femininity))
    for i = 1:length(femininity)
        hur_pred[:, i] = rand(Poisson(λ[i]), n)
    end
    return hur_pred
end

sim_m = sim_hurricanes(p_pressure, repeat([-2], 501), -3:0.01:2, 10_000)
sim_f = sim_hurricanes(p_pressure, repeat([2], 501), -3:0.01:2, 10_000)

simm_q = mapslices(col -> quantile(col, [0.05, 0.5, 0.95]), sim_m; dims=1)
simf_q = mapslices(col -> quantile(col, [0.05, 0.5, 0.95]), sim_f; dims=1)

std_pressure = stdz(dat.min_pressure)
plot(-3:0.01:2, simm_q[2, :], linewidth=3, ribbon=(simm_q[2, :] - simm_q[1, :], simm_q[3, :] - simm_q[2, :]), alpha=0.5, label="Masculine Storms (Simulation)", color=:blue, xlabel="Standardized Minimum Pressure", ylabel="Deaths")
plot!(-3:0.01:2, simf_q[2, :], linewidth=3, ribbon=(simf_q[2, :] - simf_q[1, :], simf_q[3, :] - simf_q[2, :]), alpha=0.5, label="Feminine Storms (Simulation)", color=:red)
scatter!(std_pressure[dat.female .== 0.0], dat.deaths[dat.female .== 0.0], color=:blue, label="Masculine Storms (Observations)")
scatter!(std_pressure[dat.female .== 1.0], dat.deaths[dat.female .== 1.0], color=:red, label="Feminine Storms (Observations)")

This model fails to capture the overdispersion of the observations,
which might be due to the Poisson specification[1]. But the effect size
seems pretty extreme: the model predicts roughly a 50% increase in
deaths only based on the femininity of the hurricane’s name.

The effect seems to be driven almost entirely by three outlying storms
with extremely high deaths relative to the minimum pressure, but for the
vast majority of storms, the observations are tightly clustered with
respect to deaths regardless of the gender of the hurricane’s name. That
the outlying storms are all female is more plausible as random
variation, as there are more than twice as many female-named storms in
the dataset, so the odds that some of them will be stronger than
male-named storms is nontrivial.

If we were interested in treating this hypothesis seriously, from a
purely statistical perspective, we might try some other model
specifications (negative binomial, adding in other predictors, etc). But
really, the underlying hypothesis needs more thought: one might imagine
doing more (qualitative as well as quantitative) social science research
to understand the thought process people go through when deciding how
seriously to take a hurricane warning, and where names enter into the
process. As we’ve discussed in class, **the causes of the data cannot be
found in the data**, but this analysis assumes that we can draw causal
explanations just from the statistics.

## References

[1] To be fair, the original paper *did* use a negative binomial.